# Functions to plot/process/calculate things for offline results

In [1]:
import os
import numpy as np
import time
from src.ml_io import write_netcdf_nn
import src.ml_io as ml_io
import src.ml_load as ml_load
# import src.ml_train as ml_train
import src.ml_train_nn as ml_train
import math
import pandas as pd
from sklearn.metrics import r2_score
import pickle
# import matplotlib as plt
import matplotlib.pyplot as plt

import torch
from torch.autograd import Variable
import torch.nn.functional as F
import torch.utils.data as Data
import torchvision
from torch import nn, optim
from time import time

from sklearn.metrics import r2_score

from torch_lr_finder import LRFinder
from netCDF4 import Dataset
import netCDF4
from scipy.stats import pearsonr
from scipy import stats


# %matplotlib inline  
# # from fastai.tabular import *


  pip install torch-lr-finder -v --global-option="amp"


In [ ]:
def normalized_data_1D(loaded,input_vars, indices_inputs,output_vars,in_mean,in_std,out_mean,out_std):
    # When I want to do column by column and not 2x2
    idx = 0
    dict_inputs = dict()
    for var in input_vars:
#         if len(loaded[var].shape) = 4
        dict_inputs[var] = (loaded[var]- in_mean[int(indices_inputs[idx]):int(indices_inputs[idx+1]),None,None,None])/in_std[int(indices_inputs[idx]):int(indices_inputs[idx+1]),None,None,None]
        idx = idx  + 1
    
    input_vec_3D = np.concatenate((dict_inputs['Tin'][:,:,:,:],
                                   dict_inputs['qin'][:,:,:,:],
                                   dict_inputs['uin'][:,:,:,:],
                                   dict_inputs['vin'][:,:,:,:],
                                   dict_inputs['dist'][:,:,:,:]),axis=0)
    
    dict_outputs = dict()
    idx2 = 0
    for var in output_vars:
        dict_inputs[var] = (loaded[var] - out_mean[idx2])/out_std[idx2]
        idx2 = idx2 +1
    output_vec_3D = np.concatenate((dict_inputs[output_vars[0]][:,:,:,:],
                                 dict_inputs[output_vars[1]][:,:,:,:],
                                 dict_inputs[output_vars[2]][:,:,:,:],
                                 dict_inputs[output_vars[3]][None,:,:,:],
                                 dict_inputs[output_vars[4]][None,:,:,:]),axis=0)
    
    
            
    
    return np.float32(input_vec_3D), np.float32(output_vec_3D)

In [5]:

def evaluate(model, test_features, test_labels):
    predictions = model.predict(test_features)
    errors = abs(predictions - test_labels)
    mape = 100 * np.mean(errors / abs(test_labels / 2 + predictions / 2))
    accuracy = 100 - mape
    print('Model Performance')
    print('Average Error: {:0.4f} degrees.'.format(np.mean(errors)))
    print('Accuracy = {:0.2f}%.'.format(accuracy))
    return accuracy

def PreprocessData(f_ppi, f, o_ppi, o, pp_str, n_trn_exs, z):
    """Transform data according to input preprocessor requirements and make
    make preprocessor string for saving"""
    f_pp = ml_load.init_pp(f_ppi, f)
    f = ml_load.transform_data(f_ppi, f_pp, f, z)
    o_pp = ml_load.init_pp(o_ppi, o)
    o = ml_load.transform_data(o_ppi, o_pp, o, z)
    # Make preprocessor string for saving
    pp_str = pp_str + 'F-' + f_ppi['name'] + '_'
    pp_str = pp_str + 'O-' + o_ppi['name'] + '_'
    # Add number of training examples to string
    pp_str = pp_str + 'Ntrnex' + str(n_trn_exs) + '_'
    return f_pp, f, o_pp, o, pp_str



In [6]:
def calculate_mean_std(loaded,input_vars, output_vars):
    # loaded - npz file
    mean_array_in = np.zeros(0)
    std_array_in = np.zeros(0)
    mean_array_out = np.zeros(0)
    std_array_out = np.zeros(0)
    loop = 0
    for var in input_vars:
        temp = loaded[var]
        if len(temp.shape) == 3:
                input_mean = np.atleast_1d(np.array(np.mean(np.reshape(temp,(-1)),axis=0,dtype=np.float64)))
                input_std = np.atleast_1d(np.std(np.reshape(temp,(-1)),axis=0,dtype=np.float64))
                mean_array_in = np.concatenate((mean_array_in,input_mean), axis=0)
                std_array_in = np.concatenate((std_array_in,input_std), axis=0)             
        elif len(temp.shape) == 4:
            input_mean = np.mean(np.reshape(temp,(temp.shape[0],-1)),axis=1,dtype=np.float64)
            input_std = np.std(np.reshape(temp,(temp.shape[0],-1)),axis=1,dtype=np.float64)
            mean_array_in = np.concatenate((mean_array_in,input_mean), axis=0)
            std_array_in = np.concatenate((std_array_in,input_std), axis=0)
        else:
            raise TypeError('What is the size of the variable?')
        loop = 1

    for var in output_vars:
        temp = loaded[var]
        input_mean = np.atleast_1d(np.array(np.mean(np.reshape(temp,(-1)),axis=0,dtype=np.float64)))
        input_std = np.atleast_1d(np.std(np.reshape(temp,(-1)),axis=0,dtype=np.float64))
        mean_array_out = np.concatenate((mean_array_out,input_mean), axis=0)
        std_array_out = np.concatenate((std_array_out,input_std), axis=0)  
    return mean_array_in,std_array_in,mean_array_out,std_array_out
            
    
    

In [7]:
def calculate_mean_std_dict(loaded,input_vars, output_vars):
    # loaded - npz file
    dict_in_mean = dict()
    dict_in_std = dict()
    dict_out_mean = dict()
    dict_out_std = dict()
    for var in input_vars:
        temp = loaded[var]
        if len(temp.shape) == 3:
                dict_in_mean[var] = np.atleast_1d(np.array(np.mean(np.reshape(temp,(-1)),axis=0,dtype=np.float64)))
                dict_in_std[var] = np.atleast_1d(np.std(np.reshape(temp,(-1)),axis=0,dtype=np.float64))                
        elif len(temp.shape) == 4:
            dict_in_mean[var] = np.mean(np.reshape(temp,(temp.shape[0],-1)),axis=1,dtype=np.float64)
            dict_in_std[var] = np.std(np.reshape(temp,(temp.shape[0],-1)),axis=1,dtype=np.float64)
        else:
            raise TypeError('What is the size of the variable?')

    for var in output_vars:
        temp = loaded[var]
        dict_out_mean[var] = np.atleast_1d(np.array(np.mean(np.reshape(temp,(-1)),axis=0,dtype=np.float64)))
        dict_out_std[var] = np.atleast_1d(np.std(np.reshape(temp,(-1)),axis=0,dtype=np.float64))
    return dict_in_mean, dict_in_std, dict_out_mean, dict_out_std
            
    
    

In [8]:
def normalized_data(loaded,input_vars, output_vars,in_mean,in_std,out_mean,out_std):
    idx = 0
    dict_inputs = dict()
    for var in input_vars:
#         if len(loaded[var].shape) = 4
        dict_inputs[var] = (loaded[var]- in_mean[var][:,None,None,None])/in_std[var][:,None,None,None]
        idx = idx  + 1
#     for i in dict_inputs:
#         print(dict_inputs[i].shape)
    
    input_vec_3D = np.concatenate((dict_inputs['Tin'][:,0:-1,0:-1,:],
                                   dict_inputs['qin'][:,0:-1,0:-1,:],
                                   dict_inputs['uin'][:,0:-1,1:,:],
                                   dict_inputs['vin'][:,1:,0:-1,:],
                                   dict_inputs['dist'][:,0:-1,0:-1,:]),axis=0)
    
    dict_outputs = dict()
    for var in output_vars:
        dict_inputs[var] = (loaded[var] - out_mean[var])/out_std[var]
#         print(np.mean(dict_inputs[var]))
#         print(np.std(dict_inputs[var]))
    output_vec_3D = np.concatenate((dict_inputs[output_vars[0]][:,1:-1,1:-1,:],
                                 dict_inputs[output_vars[1]][:,1:-1,1:-1,:],
                                 dict_inputs[output_vars[2]][:,1:-1,1:-1,:],
                                 dict_inputs[output_vars[3]][None,1:-1,1:-1,:],
                                 dict_inputs[output_vars[4]][None,1:-1,1:-1,:]),axis=0)
    
    
            
    
    return np.float32(input_vec_3D), np.float32(output_vec_3D)

In [9]:
def inv_normalized_outputs(data,output_vars,output_vert_dim,out_mean,out_std):
    np_output_vert_dim = np.asarray(output_vert_dim)
    inv_data = np.zeros(data.shape)
    indices = np.zeros(len(output_vert_dim)+1)
    for ind, val in zip(range(len(output_vert_dim)),output_vert_dim):
        indices[ind + 1] = indices[ind] + val

    for ind,var in zip(range(len(output_vert_dim)),output_vars):
        inv_data[:,int(indices[ind]):int(indices[ind+1])] = data[:,int(indices[ind]):int(indices[ind+1])] * out_std[var] + out_mean[var]
        ind = ind + 1
    return inv_data
        
    

In [10]:
# Schems - 
# 1) upload 3D data
# 2) Calculate input (I think here it should be column wise...) and output mean and std. -Maybe enough to use a single file for this
# 3) Transform data to have STD = 1 etc. 
# 4) Build 2x2 quanteties ?
# 5) Train on several files (Probably should give a list of files to train on) and compare to Validation



In [11]:
os.chdir('/glade/u/home/janniy/convection_parametrization/paul_codes/ML-convection_sam_flex_io/src')


In [12]:
# import sys

# # These are the usual ipython objects, including this one you are creating
# ipython_vars = ['In', 'Out', 'exit', 'quit', 'get_ipython', 'ipython_vars']

# # Get a sorted list of the objects and their sizes
# sorted([(x, sys.getsizeof(globals().get(x))) for x in dir() if not x.startswith('_') and x not in sys.modules and x not in ipython_vars], key=lambda x: x[1], reverse=True)


In [13]:
def save_mean_std(path,in_mean,in_std,out_mean,out_std):
    mean_std_list = list()
    mean_std_list.append(in_mean)
    mean_std_list.append(in_std)
    mean_std_list.append(out_mean)
    mean_std_list.append(out_std)
    pickle.dump(mean_std_list, open(path, 'wb'))
                        
    
    

In [14]:
# input_vec, output_vec = normalized_data(loaded,input_vert_vars, output_vert_vars,in_mean,in_std,out_mean,out_std)

In [15]:
def prepare_input_samples(input_vec,hor_size_x = 2, hor_size_y = 2):
#     hor_size_x = hor_si_x # domain I am interested
#     hor_size_y = hor_si_y # domain I am interestedvi
    samp_levels = input_vec.shape[0]
    if hor_size_x == 1:
        samp_X = np.int(input_vec.shape[2])
        samp_Y = np.int(input_vec.shape[1])
    else:
        samp_X = np.int(input_vec.shape[2]+1-(hor_size_x-1)*2) # np.int(input_vec.shape[2]-(hor_size_x-1)*2) #
        samp_Y = np.int(input_vec.shape[1]+1-(hor_size_y-1)*2) #
    samp_tot = samp_X *samp_Y *input_vec.shape[3] 
    Xin_scl = np.zeros((input_vec.shape[0],hor_size_x,hor_size_y,samp_tot))

    ind_sample = 0
    for ti in range(input_vec.shape[3]):
        for j in range(samp_Y):
            for i in range(samp_X):
                Xin_scl[:,:,:,ind_sample] = input_vec[:,j:j+hor_size_y,i:i+hor_size_x,ti]
                ind_sample = ind_sample + 1
    return Xin_scl

In [17]:
def reshape_output_from_3D(output_vec):
    output_vec = np.moveaxis(output_vec, (3),(1))
    output_vec = np.reshape(output_vec,(output_vec.shape[0],-1))
    return output_vec

In [18]:
def inv_reshape_output_to_3D(output_vec,shape_3D):
    # output_vec - assume 2D (samples, levels)
    # shape_3D - I think that at the moment should be time,lat -2 , lon -2 (the -2 depending on size of network in the horizontal)
    output_vec_3D = np.moveaxis(output_vec,0,1)
    output_vec_3D = np.reshape(output_vec_3D,(output_vec_3D.shape[0],shape_3D[0],shape_3D[1],shape_3D[2]))
    return output_vec_3D

# I need to write the inverse of the reshaping procedure

In [20]:
def y_x_to_pyorch(inputs, outputs):
    X_py = torch.from_numpy(inputs.reshape(-1,inputs.shape[1],inputs.shape[2],inputs.shape[3])).float()
    y_train_py = torch.from_numpy(outputs.reshape(-1,outputs.shape[1])).float()
    return X_py, y_train_py

In [21]:
def to_databunch(X_py, y_train_py,BATCH_SIZE  = 1024):
    BATCH_SIZE =  1024 # 128
    torch_dataset = Data.TensorDataset(X_py, y_train_py)

    loader = Data.DataLoader(
        dataset=torch_dataset,
        batch_size=BATCH_SIZE,
        shuffle=True, num_workers=4,)
    return loader, torch_dataset 


In [22]:
def save_training_file(path,x,y):
    np.savez_compressed(path,
                        x=x,
                        y=y)
                        
    
    

In [23]:

class Net_fully(nn.Module):
    def __init__(self,hor_size_x = 2):
        super(Net_fully, self).__init__()
        self.hor_size = hor_size_x*hor_size_x
        self.f_size = 157 #48*4+1
        self.n_size = 128
        self.linear1 = nn.Linear(self.f_size*self.hor_size, self.n_size)
        self.linear2 = nn.Linear(self.n_size, self.n_size)
        self.linear3 = nn.Linear(self.n_size, self.n_size)
        self.linear4 = nn.Linear(self.n_size, self.n_size)
        self.linear5 = nn.Linear(self.n_size, 111)
    def forward(self, x):
#         x = self.dense0_bn(x)
        x = torch.reshape(x,(x.shape[0],-1))
        x = F.relu(self.linear1(x))
        x = F.relu(self.linear2(x))
        x = F.relu(self.linear3(x))
        x = F.relu(self.linear4(x))
        x = self.linear5(x)
        return x

In [24]:
def rmse(x,y): return math.sqrt(((x-y)**2).mean())

In [25]:
def train_model(net,criterion,trainloader,optimizer,batchsize):
    net.train()
    test_loss = 0
    for step, (batch_x, batch_y) in enumerate(trainloader):  # for each training step
        b_x = Variable(batch_x)
        b_y = Variable(batch_y)
        prediction = net(b_x)  # input x and predict based on x
        loss = criterion(prediction, b_y)  # must be (1. nn output, 2. target)
        optimizer.zero_grad()  # clear gradients for next train
        loss.backward()  # backpropagation, compute gradients
        optimizer.step()  # apply gradients
        test_loss = test_loss + loss.data.numpy()
    print('the loss in this Epoch',test_loss)

In [26]:
def test_model3D(net,X_train_val_norm_py,y_train_val):
    net.eval()
    pred_val = net(Variable(X_train_val_norm_py))
    pred_val_np = pred_val.data.numpy()
    print('RMSE: ',rmse(pred_val_np,y_train_val), ' R2:' ,r2_score(np.reshape(y_train_val,(y_train_val.shape[0],-1)), np.reshape(pred_val_np,(pred_val_np.shape[0],-1)),multioutput='variance_weighted'))
    return r2_score(np.reshape(y_train_val,(y_train_val.shape[0],-1)), np.reshape(pred_val_np,(pred_val_np.shape[0],-1)),multioutput='variance_weighted')

In [27]:
def test_model(net,X_train_val_norm_py,y_train_val):
    net.eval()
    pred_val = net(Variable(X_train_val_norm_py))
    print('RMSE: ',rmse(pred_val.data.numpy(),y_train_val), ' R2:' ,r2_score(y_train_val[:,:], pred_val.data.numpy()[:,:],multioutput='variance_weighted'))
    return r2_score(y_train_val[:,:], pred_val.data.numpy()[:,:],multioutput='variance_weighted')

In [28]:
def test_model_per_var(net,X_train_val_norm_py,y_train_val, output_vert_dim=[48]):
    net.eval()
    pred_val = net(Variable(X_train_val_norm_py))
    j = 0
    for i in output_vert_dim:
        print('R2:' ,r2_score(y_train_val[:,j:j+i], pred_val.data.numpy()[:,j:j+i],multioutput='variance_weighted'))
        j = j + i
    return r2_score(y_train_val[:,:], pred_val.data.numpy()[:,:],multioutput='variance_weighted')

In [29]:
def train_model_cyclic(net,criterion,trainloader,optimizer,batchsize,scheduler):
#     amp.initialize(model, optimizer, opt_level)
    net.train()
    test_loss = 0
    for step, (batch_x, batch_y) in enumerate(trainloader):  # for each training step
        b_x = Variable(batch_x)
        b_y = Variable(batch_y)
        prediction = net(b_x)  # input x and predict based on x
        loss = criterion(prediction, b_y)  # must be (1. nn output, 2. target)
        optimizer.zero_grad()  # clear gradients for next train
        loss.backward()  # backpropagation, compute gradients
#         with amp.scale_loss(loss,optimizer) as scaled_loss:
#                     scaled_loss.backward()                         # backprop            
        optimizer.step()  # apply gradients
        optimizer.zero_grad()
        test_loss = test_loss + loss.data.numpy()
        scheduler.step() #defined I think on the epoches...
    print('the loss in this Epoch',test_loss)

In [30]:
def build_validation(path_val):
    loaded_val = np.load(path_val)
    X_val = loaded_val['x']
    y_val = loaded_val['y']
    X_py_val, y_train_py_val= y_x_to_pyorch(np.moveaxis(X_val,3,0),np.moveaxis(y_val,1,0))
    loader, torch_dataset = to_databunch(X_py_val, y_train_py_val)
    return X_py_val, y_train_py_val,loader, torch_dataset

In [31]:
def coarse_var(variable,coarse_fact = 4):
    # I assume we coarse over dims 2,3
    X_size = int(variable.shape[3]/coarse_fact)
    Y_size = int(variable.shape[2]/coarse_fact)
    var_coarse = np.zeros((variable.shape[0],variable.shape[1],Y_size,X_size))
    for j in range(Y_size):
        for i in range(X_size):
            var_coarse[:,:,j,i] = np.mean(variable[:,:,j*coarse_fact:(j+1)*coarse_fact,i*coarse_fact:(i+1)*coarse_fact],axis = (2,3))
            
    return var_coarse

In [32]:
def r2_2D(true,pred):
    #assume dims are levs,time,y,x
    r2_2d = np.zeros((pred.shape[2],pred.shape[0]))
    for lev in range(pred.shape[0]):
        for yi in range(pred.shape[2]):
            r2_2d[yi,lev] = r2_score(np.reshape(true[lev,:,yi,:],-1),np.reshape(pred[lev,:,yi,:],-1))
    return r2_2d

In [33]:

class Net_fully_1x1(nn.Module):
    def __init__(self,hor_size_x = 1):
        super(Net_fully_1x1, self).__init__()
        self.f_size = 157 #48*4+1
        self.n_size = 128
        self.linear1 = nn.Linear(self.f_size, self.n_size)
        self.linear2 = nn.Linear(self.n_size, self.n_size)
        self.linear3 = nn.Linear(self.n_size, self.n_size)
        self.linear4 = nn.Linear(self.n_size, self.n_size)
        self.linear5 = nn.Linear(self.n_size, 111)
    def forward(self, x):
#         x = self.dense0_bn(x)
        x = F.relu(self.linear1(x))
        x = F.relu(self.linear2(x))
        x = F.relu(self.linear3(x))
        x = F.relu(self.linear4(x))
        x = self.linear5(x)
        return x
    

In [34]:

class Net_fully_1x1_3_layer(nn.Module):
    def __init__(self,hor_size_x = 1):
        super(Net_fully_1x1_3_layer, self).__init__()
        self.f_size = 157 #48*4+1
        self.n_size = 128
        self.linear1 = nn.Linear(self.f_size, self.n_size)
        self.linear2 = nn.Linear(self.n_size, self.n_size)
        self.linear3 = nn.Linear(self.n_size, 111)
    def forward(self, x):
#         x = self.dense0_bn(x)
        x = F.relu(self.linear1(x))
        x = F.relu(self.linear2(x))
        x = self.linear3(x)
        return x
    

In [35]:
def build_validation_1D(path_val):
    loaded_val = np.load(path_val)
    X_val = loaded_val['x']
    y_val = loaded_val['y']
    X_py_val, y_train_py_val= y_x_to_pyorch(np.moveaxis(X_val,3,0),np.moveaxis(y_val,1,0))
    loader, torch_dataset = to_databunch(X_py_val[:,:,0,0], y_train_py_val)
    return X_py_val, y_train_py_val,loader, torch_dataset

### Plot function

In [36]:
def plot_R2_contourf(Field,indices,p,save=0,save_name = 'del'):
    
    letters = ["(a)","(b)","(c)","(d)","(e)",'(f)']
    
    
    fontsize = 10
    fontsize_legend = 8
    h_width = 0.1
    v_width = 0.1
    h_width_cbar = 0.02
    colorbar_width =0.02

    row_dim = 3
    col_dim = 2

    fig_witdth = 0.7
    fig_height = 0.5
    fig_h_dist = 0.4
    fig_v_dist = 0.2
    
    size1 = 2
    fig, axes = plt.subplots(row_dim,col_dim)
    # fig.set_figheight(10)
    # fig.set_figwidth(10)
    upper_level_p = 48
    lowest_level = 0
    cmap1 = 'bwr'
    n=30
    yind1 = 65
    x_ind1 = 10
    # wnumber = 1
    ind_ind = 0
    
    
    x0 = axes[0,0].get_position().x0
    x1 = axes[0,0].get_position().x1
    y0 = axes[0,0].get_position().y0
    y1 = axes[0,0].get_position().y1

    for i in range(row_dim):
        for j  in range(col_dim):
            if i == 0 and j == 1:
                axes[i,j].set_position([x0 + j*fig_witdth + j*fig_h_dist,y0- i*fig_height - i*fig_v_dist,fig_witdth/1.5,fig_height])
            else:    
                axes[i,j].set_position([x0 + j*fig_witdth + j*fig_h_dist,y0- i*fig_height - i*fig_v_dist,fig_witdth,fig_height])
    
    
    for i in range(row_dim):
        field_temp = Field[:,int(indices[ind_ind]):int(indices[ind_ind+1])]
        range11 = np.max(field_temp)
        levels = np.linspace(-range11, range11, n+1)

    
        print(len(range(Field.shape[0])))
        print(p[0:field_temp.shape[1]].shape)
        print(field_temp.shape)
        cax_xy1 =axes[i,0].contourf(range(Field.shape[0]),p[0:field_temp.shape[1]],field_temp.T,levels = levels,cmap=cmap1)
#         axes[i, 0].set_ylim(np.amax(p[lowest_level]), np.amin(p))
#         print('yani')
        axes[i, 0].set_ylim(np.amax(p[0:field_temp.shape[1]]), np.amin(p[0:field_temp.shape[1]]))
        pos1_rsq = axes[i, 0].get_position() # get the original position 
        cbaxes_rsq = fig.add_axes([pos1_rsq.x0+pos1_rsq.width + h_width_cbar, pos1_rsq.y0, colorbar_width, fig_height]) 
        cbar_rsq = fig.colorbar(cax_xy1,cax = cbaxes_rsq, ax=axes[i,0], shrink=1.0,)
        ind_ind = ind_ind + 1

    field_temp = Field[:,int(indices[ind_ind]):int(indices[ind_ind+1])]
    cax_xy1_3 =axes[0,1].plot(field_temp)
    ind_ind  = ind_ind + 1 
    field_temp = Field[:,int(indices[ind_ind]):int(indices[ind_ind+1])]
    cax_xy2_3 =axes[1,1].plot(field_temp)

    
    ind_let = 0
    for i in range(row_dim):
        for j  in range(col_dim):
            atext2 = axes[i, j].text(axes[0,0].get_position().x0*1.5+ j*fig_witdth + j*fig_h_dist,axes[0,0].get_position().y0*0.76- (i-1)*fig_height - (i-1)*fig_v_dist ,letters[ind_let],transform = fig.transFigure,color="black",fontsize=fontsize)

    #         header= axes[i, j].text(axes[0,0].get_position().x0*1.5+ j*fig_witdth + j*fig_h_dist,axes[0,0].get_position().y0*0.92- (i-1)*fig_height - (i-1)*fig_v_dist ,headers[ind_let],transform = fig.transFigure,color="black",fontsize=fontsize)
            ind_let = ind_let + 1

    if save:
        save_path = '/glade/work/janniy/data_for_offline_momentum_paper/'
        plt.savefig(save_path+ save_name +'.eps', format='eps', bbox_inches="tight")
        
#         from matplotlib.ticker import PercentFormatter

# #         data = [1000, 1000, 5000, 3000, 4000, 16000, 2000]

# #         axes[3,0].hist(np.reshape(field1,-1), weights=np.ones(np.reshape(field1,-1).shape) / np.reshape(field1,-1).shape,bins = 1000)
# #         axes[3,1].hist(np.reshape(field2,-1), weights=np.ones(np.reshape(field2,-1).shape) / np.reshape(field2,-1).shape,bins = 1000)

# #         axes[3,0].gca().yaxis.set_major_formatter(PercentFormatter(1))
    

In [37]:
def plot_R2_contourf_production(Field,y,indices,p,save=0,save_name = 'del',coarse_flag = False,lev_Rsq = np.arange(-0.1, 0.37, 0.05),Rsq_ticks1=np.arange(-0.1, 0.4, 0.1)):
    
    letters = ["(a)","(b)","(c)","(d)","(e)",'(f)']
    headers = [r"Zonal momentum subgrid flux",r"Meridional momentum subgrid flux",r"Diffusivity",r"Zonal momentum surface subgrid flux ",r"Meridional momentum surface subgrid flux",r"diffusivity RF"]

    y_plot = (y - np.mean(y))/10**6
    ticks_lat = [-5,0,5]
    
    ticks_press_tkz = [1000,800,600]
    ticks_press = [900,600,300]

    
#     lev_Rsq = np.arange(-0.1, 0.37, 0.05)
    lev_Rsq_coarse = np.arange(-0.35, 0.85, 0.05)
    cont_min_rsq = 0
    cont_max_rsq = 1
    lev_Rsq_diffusivity= np.arange(-0.1, 1.0, 0.05)

    
    fontsize = 14
    fontsize_legend = 8
    h_width = 0.1
    v_width = 0.1
    h_width_cbar = 0.02
    colorbar_width =0.02

    p_lev_diff = 15
    
    row_dim = 3
    col_dim = 2

    fig_witdth = 0.7
    fig_height = 0.5
    fig_h_dist = 0.27
    fig_v_dist = 0.18
    
    size1 = 2
    fig, axes = plt.subplots(row_dim,col_dim)
    # fig.set_figheight(10)
    # fig.set_figwidth(10)
    upper_level_p = 48
    lowest_level = 0
    cmap1 = 'bwr'
    n=30
    yind1 = 65
    x_ind1 = 10
    # wnumber = 1
    ind_ind = 0
    
    
    x0 = axes[0,0].get_position().x0
    x1 = axes[0,0].get_position().x1
    y0 = axes[0,0].get_position().y0
    y1 = axes[0,0].get_position().y1

    for i in range(row_dim):
        for j  in range(col_dim):
            if i == 0 and j == 1:
                axes[i,j].set_position([x0 + j*fig_witdth + j*fig_h_dist,y0- i*fig_height - i*fig_v_dist,fig_witdth/1.0,fig_height])
            else:    
                axes[i,j].set_position([x0 + j*fig_witdth + j*fig_h_dist,y0- i*fig_height - i*fig_v_dist,fig_witdth,fig_height])
    
        
    field_temp = Field[:,int(indices[ind_ind+1]):int(indices[ind_ind+2])]
    i = 0
    if coarse_flag:
        levels = lev_Rsq_coarse
    else:
        levels = lev_Rsq

    cax_xy1 =axes[i,0].contourf(y_plot,p[0:field_temp.shape[1]],field_temp.T,levels = levels,extend='both')#,cmap=cmap1)
    #         axes[i, 0].set_ylim(np.amax(p[lowest_level]), np.amin(p))
    #         print('yani')
    axes[i, 0].set_ylim(np.amax(p[:]), np.amin(p[0:field_temp.shape[1]]))
    pos1_rsq = axes[i, 0].get_position() # get the original position 
    cbaxes_rsq = fig.add_axes([pos1_rsq.x0+pos1_rsq.width + h_width_cbar, pos1_rsq.y0, colorbar_width, fig_height]) 
#     cbar_rsq = fig.colorbar(cax_xy1,cax = cbaxes_rsq, ax=axes[i,0], shrink=1.0,ticks=np.arange(-0.1, 0.4, 0.1))
    if coarse_flag:
        cbar_rsq = fig.colorbar(cax_xy1,cax = cbaxes_rsq, ax=axes[i,0], shrink=1.0,ticks=np.arange(-0.25, 0.85, 0.25))
    else:
        cbar_rsq = fig.colorbar(cax_xy1,cax = cbaxes_rsq, ax=axes[i,0], shrink=1.0,ticks=Rsq_ticks1)

    cbar_rsq.ax.set_xlabel('$R^2$', fontsize=fontsize)
    cbar_rsq.ax.tick_params(labelsize=fontsize-2)
    
    ind_ind = ind_ind + 1
    field_temp = Field[:,int(indices[ind_ind+1]):int(indices[ind_ind+2])]
    i = 1 
    if coarse_flag:
        levels = lev_Rsq_coarse
    else:
        levels = lev_Rsq

    cax_xy1 =axes[0,1].contourf(y_plot,p[0:field_temp.shape[1]],field_temp.T,levels = levels,extend='both')#,cmap=cmap1)
    #         axes[i, 0].set_ylim(np.amax(p[lowest_level]), np.amin(p))
    #         print('yani')
    axes[0, 1].set_ylim(np.amax(p[:]), np.amin(p[0:field_temp.shape[1]]))
    pos1_rsq = axes[0, 1].get_position() # get the original position 
    cbaxes_rsq = fig.add_axes([pos1_rsq.x0+pos1_rsq.width + h_width_cbar, pos1_rsq.y0, colorbar_width, fig_height]) 
    if coarse_flag:
        cbar_rsq = fig.colorbar(cax_xy1,cax = cbaxes_rsq, ax=axes[0,1], shrink=1.0,ticks=np.arange(-0.25, 0.85, 0.25))
    else:
        cbar_rsq = fig.colorbar(cax_xy1,cax = cbaxes_rsq, ax=axes[0,1], shrink=1.0,ticks=Rsq_ticks1)
    ind_ind = ind_ind + 1
    cbar_rsq.ax.set_xlabel('$R^2$', fontsize=fontsize)
    cbar_rsq.ax.tick_params(labelsize=fontsize-2)
    
    ind_ind = 0
    field_temp = Field[:,int(indices[ind_ind]):int(indices[ind_ind+1])]
    i = 1 
    levels = lev_Rsq_diffusivity
    cax_xy1 =axes[i,0].contourf(y_plot,p[0:field_temp.shape[1]],field_temp.T,levels = levels,extend='both')#,cmap=cmap1)
    #         axes[i, 0].set_ylim(np.amax(p[lowest_level]), np.amin(p))
    #         print('yani')
    axes[i, 0].set_ylim(np.amax(p[:]), np.amin(p[:]))
    pos1_rsq = axes[i, 0].get_position() # get the original position 
    cbaxes_rsq = fig.add_axes([pos1_rsq.x0+pos1_rsq.width + h_width_cbar, pos1_rsq.y0, colorbar_width, fig_height]) 
    cbar_rsq = fig.colorbar(cax_xy1,cax = cbaxes_rsq, ax=axes[i,0], shrink=1.0,ticks=np.arange(0, 1, 0.25))
    ind_ind = ind_ind + 1
    cbar_rsq.ax.set_xlabel('$R^2$', fontsize=fontsize)
    cbar_rsq.ax.tick_params(labelsize=fontsize-2)
    
    
    ind_ind = 3
    field_temp = Field[:,int(indices[ind_ind]):int(indices[ind_ind+1])]
    cax_xy1_3 =axes[1,1].plot(y_plot,field_temp)
    ind_ind  = ind_ind + 1 
    field_temp = Field[:,int(indices[ind_ind]):int(indices[ind_ind+1])]
    cax_xy2_3 =axes[2,0].plot(y_plot,field_temp)

    
    ind_let = 0
    for i in range(row_dim):
        for j  in range(col_dim):
            atext2 = axes[i, j].text(axes[0,0].get_position().x0*0.9+ j*fig_witdth + j*fig_h_dist,axes[0,0].get_position().y0*0.76- (i-1)*fig_height - (i-1)*fig_v_dist ,letters[ind_let],transform = fig.transFigure,color="black",fontsize=fontsize)

            header= axes[i, j].text(axes[0,0].get_position().x0*1.3+ j*fig_witdth + j*fig_h_dist,axes[0,0].get_position().y0*0.76- (i-1)*fig_height - (i-1)*fig_v_dist ,headers[ind_let],transform = fig.transFigure,color="black",fontsize=fontsize)
            ind_let = ind_let + 1

            
            
            
    axes[0, 0].set_xlim(np.amin(y_plot), np.amax(y_plot))
    axes[1, 0].set_xlim(np.amin(y_plot), np.amax(y_plot))
    axes[2, 0].set_xlim(np.amin(y_plot), np.amax(y_plot))
    axes[0, 1].set_xlim(np.amin(y_plot), np.amax(y_plot))
    axes[1, 1].set_xlim(np.amin(y_plot), np.amax(y_plot))
    


    axes[0, 0].set_ylabel('Pressure [hPa]',fontsize=fontsize)
    axes[1, 0].set_ylabel('Pressure [hPa]',fontsize=fontsize)
    axes[2, 0].set_ylabel(r'R$^2$',fontsize=fontsize)
    axes[1, 1].set_ylabel(r'R$^2$',fontsize=fontsize)


# axes[2, 0].set_xlabel('y [$10^3$ km]',fontsize=fontsize)
# axes[1, 1].set_xlabel('y [$10^3$ km]',fontsize=fontsize)


    axes[2, 0].set_xlabel('Latitude',fontsize=fontsize)
    axes[1, 1].set_xlabel('Latitude',fontsize=fontsize)
    
    fontsize_ticks = fontsize -2
    ticks_press_R2 = [0.25,0.5,0.75]    
    ticks_lat_degrees = ['$-45^\circ$','$0^\circ$','$45^\circ$']
    axes[1, 1].set_yticks(ticks_press_R2)
    axes[1, 1].set_yticklabels(ticks_press_R2,fontsize=fontsize_ticks)
    axes[2, 0].set_yticks(ticks_press_R2)
    axes[2, 0].set_yticklabels(ticks_press_R2,fontsize=fontsize_ticks)

    axes[2, 0].set_xticks(ticks_lat)
    axes[2, 0].set_xticklabels(ticks_lat_degrees,fontsize=fontsize_ticks)
    axes[1, 1].set_xticks(ticks_lat)
    axes[1, 1].set_xticklabels(ticks_lat_degrees,fontsize=fontsize_ticks)
     
    axes[0, 0].set_xticks(ticks_lat)
    axes[0, 0].set_xticklabels([])
    axes[1, 0].set_xticks(ticks_lat)
    axes[1, 0].set_xticklabels([])
    axes[0, 1].set_xticks(ticks_lat)
    axes[0, 1].set_xticklabels([])
    
    axes[0, 0].set_yticks(ticks_press)
    axes[0, 0].set_yticklabels(ticks_press,fontsize=fontsize_ticks)
    
    axes[1, 0].set_yticks(ticks_press)
    axes[1, 0].set_yticklabels(ticks_press,fontsize=fontsize_ticks)

    axes[0, 1].set_yticks(ticks_press)
    axes[0, 1].set_yticklabels([])

    
    
#     axes[2, 0].set_xticks(ticks_lat)
#     axes[2, 0].set_xticklabels([])

    fig.delaxes(axes[2,1])
        
        
    if save:
        save_path = '/glade/work/janniy/data_for_offline_momentum_paper/plots/'
        plt.savefig(save_path+ save_name +'.eps', format='eps', bbox_inches="tight")
        
#         from matplotlib.ticker import PercentFormatter

# #         data = [1000, 1000, 5000, 3000, 4000, 16000, 2000]

# #         axes[3,0].hist(np.reshape(field1,-1), weights=np.ones(np.reshape(field1,-1).shape) / np.reshape(field1,-1).shape,bins = 1000)
# #         axes[3,1].hist(np.reshape(field2,-1), weights=np.ones(np.reshape(field2,-1).shape) / np.reshape(field2,-1).shape,bins = 1000)

# #         axes[3,0].gca().yaxis.set_major_formatter(PercentFormatter(1))
    

In [38]:
def calc_lapse_rate(T,z,p,min_lev_trop_ind=10):
    # Calculating the index of tropopause height
    d_z1 = np.zeros(T.shape[0]-2)
    d_z1[:] = z[2:] - z[:-2]
    T_zonal_t_mean = np.mean(T,axis=(2,3))
    T_zonal_t_mean_diff = T_zonal_t_mean[2:,:]-T_zonal_t_mean[:-2,:]
    dT_dz = T_zonal_t_mean_diff[:,:]/d_z1[:,None]

#     for yi in range(field.shape[1]):
#         np.where(field[:,yi] > -0.002)
    trop_ind = np.zeros(T.shape[1])
    trop_z = np.zeros(T.shape[1])
    trop_p = np.zeros(T.shape[1])
    for yi in range(dT_dz.shape[1]):
        aa = np.where(dT_dz[min_lev_trop_ind:,yi] > -0.002)
        trop_ind[yi] = min(aa)[0]  + min_lev_trop_ind 
        trop_z[yi] = z[min(aa)[0]+min_lev_trop_ind ]
        trop_p[yi] = p[min(aa)[0]+min_lev_trop_ind ]
        
    return trop_ind,trop_z,trop_p

In [40]:
def plot_mean_contourf_production(Field,y,dz,indices,p,save=0,save_name = 'del',coarse_flag = False):
    
    letters = ["(a)","(b)","(c)","(d)","(e)",'(f)']
    headers = [r"Zonal momentum subgrid flux",r"Meridional momentum subgrid flux",r"Diffusivity",r"Zonal momentum surface subgrid flux ",r"Meridional momentum surface subgrid flux",r"diffusivity RF"]
#     headers = [r"Zonal momentum subgrid flux",r"Meridional momentum subgrid flux",r"Diffusivity",r"Energy surface subgrid flux ",r"Moisture surface subgrid flux",r"diffusivity RF"]

    y_plot = (y - np.mean(y))/10**6
    ticks_lat = [-5,0,5]
    
    ticks_press_tkz = [1000,800,600]
    ticks_press = [900,600,300]

    
    lev_Rsq = np.arange(-0.04, 0.041, 0.002) 
#     lev_Rsq = np.arange(-0.01, 0.01, 0.001) 
    
    lev_Rsq_ticks = np.arange(-0.03, 0.031, 0.02) 
#     lev_Rsq_ticks = np.arange(-0.01, 0.011, 0.01) 

    ticks_press_R2 = [-0.006,-0.003, 0,0.003,0.006] #np.arange(-0.0075, 0.00751, 0.005)
    
    cont_min_rsq = 0
    cont_max_rsq = 1
    lev_Rsq_diffusivity= np.arange(0, 26.0, 1)
    lev_Rsq_diffusivity_ticks= np.arange(0, 26.0, 5)

    
    fontsize = 14
    fontsize_legend = 8
    h_width = 0.1
    v_width = 0.1
    h_width_cbar = 0.02
    colorbar_width =0.02

    p_lev_diff = 15
    
    row_dim = 3
    col_dim = 2

    fig_witdth = 0.7
    fig_height = 0.5
    fig_h_dist = 0.27
    fig_v_dist = 0.18
    
    size1 = 2
    fig, axes = plt.subplots(row_dim,col_dim)
    # fig.set_figheight(10)
    # fig.set_figwidth(10)
    upper_level_p = 48
    lowest_level = 0
    cmap1 = 'bwr'
    n=30
    yind1 = 65
    x_ind1 = 10
    # wnumber = 1
    ind_ind = 0
    
    
    x0 = axes[0,0].get_position().x0
    x1 = axes[0,0].get_position().x1
    y0 = axes[0,0].get_position().y0
    y1 = axes[0,0].get_position().y1

    for i in range(row_dim):
        for j  in range(col_dim):
            if i == 0 and j == 1:
                axes[i,j].set_position([x0 + j*fig_witdth + j*fig_h_dist,y0- i*fig_height - i*fig_v_dist,fig_witdth/1.0,fig_height])
            else:    
                axes[i,j].set_position([x0 + j*fig_witdth + j*fig_h_dist,y0- i*fig_height - i*fig_v_dist,fig_witdth,fig_height])
    
        
    field_temp = Field[:,int(indices[ind_ind+1]):int(indices[ind_ind+2])]
    i = 0
    if coarse_flag:
        levels = lev_Rsq_coarse
    else:
        levels = lev_Rsq

    cax_xy1 =axes[i,0].contourf(y_plot,p[0:field_temp.shape[1]],field_temp.T,levels = levels,cmap=cmap1,extend='both')
    #         axes[i, 0].set_ylim(np.amax(p[lowest_level]), np.amin(p))
    #         print('yani')
    axes[i, 0].set_ylim(np.amax(p[:]), np.amin(p[0:field_temp.shape[1]]))
    pos1_rsq = axes[i, 0].get_position() # get the original position 
    cbaxes_rsq = fig.add_axes([pos1_rsq.x0+pos1_rsq.width + h_width_cbar, pos1_rsq.y0, colorbar_width, fig_height]) 
#     cbar_rsq = fig.colorbar(cax_xy1,cax = cbaxes_rsq, ax=axes[i,0], shrink=1.0,ticks=np.arange(-0.1, 0.4, 0.1))
    if coarse_flag:
        cbar_rsq = fig.colorbar(cax_xy1,cax = cbaxes_rsq, ax=axes[i,0], shrink=1.0,ticks=np.arange(-0.25, 0.85, 0.25))
    else:
        cbar_rsq = fig.colorbar(cax_xy1,cax = cbaxes_rsq, ax=axes[i,0], shrink=1.0,ticks=lev_Rsq_ticks)

    cbar_rsq.ax.set_xlabel(r'kg m$^{-1}$ s$^{-2}$', fontsize=fontsize)
    cbar_rsq.ax.tick_params(labelsize=fontsize-2)
    
    ind_ind = ind_ind + 1
    field_temp = Field[:,int(indices[ind_ind+1]):int(indices[ind_ind+2])]
    i = 1 
    if coarse_flag:
        levels = lev_Rsq_coarse
    else:
        levels = lev_Rsq

    cax_xy1 =axes[0,1].contourf(y_plot,p[0:field_temp.shape[1]],field_temp.T,levels = levels,cmap=cmap1,extend='both')
    #         axes[i, 0].set_ylim(np.amax(p[lowest_level]), np.amin(p))
    #         print('yani')
    axes[0, 1].set_ylim(np.amax(p[:]), np.amin(p[0:field_temp.shape[1]]))
    pos1_rsq = axes[0, 1].get_position() # get the original position 
    cbaxes_rsq = fig.add_axes([pos1_rsq.x0+pos1_rsq.width + h_width_cbar, pos1_rsq.y0, colorbar_width, fig_height]) 
    if coarse_flag:
        cbar_rsq = fig.colorbar(cax_xy1,cax = cbaxes_rsq, ax=axes[0,1], shrink=1.0,ticks=np.arange(-0.25, 0.85, 0.25))
    else:
        cbar_rsq = fig.colorbar(cax_xy1,cax = cbaxes_rsq, ax=axes[0,1], shrink=1.0,ticks=lev_Rsq_ticks)
    ind_ind = ind_ind + 1
    cbar_rsq.ax.set_xlabel(r'kg m$^{-1}$ s$^{-2}$', fontsize=fontsize)
    cbar_rsq.ax.tick_params(labelsize=fontsize-2)
    
    ind_ind = 0
    field_temp = Field[:,int(indices[ind_ind]):int(indices[ind_ind+1])]
    i = 1 
    levels = lev_Rsq_diffusivity
    cax_xy1 =axes[i,0].contourf(y_plot,p[0:field_temp.shape[1]],field_temp.T,levels = levels,cmap='Reds',extend='both')
    #         axes[i, 0].set_ylim(np.amax(p[lowest_level]), np.amin(p))
    #         print('yani')
    axes[i, 0].set_ylim(np.amax(p[:]), np.amin(p[:]))
    pos1_rsq = axes[i, 0].get_position() # get the original position 
    cbaxes_rsq = fig.add_axes([pos1_rsq.x0+pos1_rsq.width + h_width_cbar, pos1_rsq.y0, colorbar_width, fig_height]) 
    cbar_rsq = fig.colorbar(cax_xy1,cax = cbaxes_rsq, ax=axes[i,0], shrink=1.0,ticks=np.arange(0, 25, 5))
    ind_ind = ind_ind + 1
    cbar_rsq.ax.set_xlabel(r'm$^2$ s$^{-1}$', fontsize=fontsize)
    cbar_rsq.ax.tick_params(labelsize=fontsize-2)
    
    
    ind_ind = 3
    field_temp = Field[:,int(indices[ind_ind]):int(indices[ind_ind+1])]*dz
    cax_xy1_3 =axes[1,1].plot(y_plot,field_temp)
    ind_ind  = ind_ind + 1 
    field_temp = Field[:,int(indices[ind_ind]):int(indices[ind_ind+1])]*dz
    cax_xy2_3 =axes[2,0].plot(y_plot,field_temp)

    
    ind_let = 0
    for i in range(row_dim):
        for j  in range(col_dim):
            atext2 = axes[i, j].text(axes[0,0].get_position().x0*0.9+ j*fig_witdth + j*fig_h_dist,axes[0,0].get_position().y0*0.76- (i-1)*fig_height - (i-1)*fig_v_dist ,letters[ind_let],transform = fig.transFigure,color="black",fontsize=fontsize)

            header= axes[i, j].text(axes[0,0].get_position().x0*1.3+ j*fig_witdth + j*fig_h_dist,axes[0,0].get_position().y0*0.76- (i-1)*fig_height - (i-1)*fig_v_dist ,headers[ind_let],transform = fig.transFigure,color="black",fontsize=fontsize)
            ind_let = ind_let + 1

            
            
            
    axes[0, 0].set_xlim(np.amin(y_plot), np.amax(y_plot))
    axes[1, 0].set_xlim(np.amin(y_plot), np.amax(y_plot))
    axes[2, 0].set_xlim(np.amin(y_plot), np.amax(y_plot))
    axes[0, 1].set_xlim(np.amin(y_plot), np.amax(y_plot))
    axes[1, 1].set_xlim(np.amin(y_plot), np.amax(y_plot))
    


    axes[0, 0].set_ylabel('Pressure [hPa]',fontsize=fontsize)
    axes[1, 0].set_ylabel('Pressure [hPa]',fontsize=fontsize)
    axes[2, 0].set_ylabel(r'kg m$^{-1}$ s$^{-2}$',fontsize=fontsize)
    axes[1, 1].set_ylabel(r'kg m$^{-1}$ s$^{-2}$',fontsize=fontsize)


# axes[2, 0].set_xlabel('y [$10^3$ km]',fontsize=fontsize)
# axes[1, 1].set_xlabel('y [$10^3$ km]',fontsize=fontsize)


    axes[2, 0].set_xlabel('Latitude',fontsize=fontsize)
    axes[1, 1].set_xlabel('Latitude',fontsize=fontsize)
    
    fontsize_ticks = fontsize -2  
    ticks_lat_degrees = ['$-45^\circ$','$0^\circ$','$45^\circ$']
    axes[1, 1].set_yticks(ticks_press_R2)
    axes[1, 1].set_yticklabels(ticks_press_R2,fontsize=fontsize_ticks)
    axes[2, 0].set_yticks(ticks_press_R2)
    axes[2, 0].set_yticklabels(ticks_press_R2,fontsize=fontsize_ticks)

    axes[2, 0].set_xticks(ticks_lat)
    axes[2, 0].set_xticklabels(ticks_lat_degrees,fontsize=fontsize_ticks)
    axes[1, 1].set_xticks(ticks_lat)
    axes[1, 1].set_xticklabels(ticks_lat_degrees,fontsize=fontsize_ticks)
     
    axes[0, 0].set_xticks(ticks_lat)
    axes[0, 0].set_xticklabels([])
    axes[1, 0].set_xticks(ticks_lat)
    axes[1, 0].set_xticklabels([])
    axes[0, 1].set_xticks(ticks_lat)
    axes[0, 1].set_xticklabels([])
    
    axes[0, 0].set_yticks(ticks_press)
    axes[0, 0].set_yticklabels(ticks_press,fontsize=fontsize_ticks)
    
    axes[1, 0].set_yticks(ticks_press)
    axes[1, 0].set_yticklabels(ticks_press,fontsize=fontsize_ticks)

    axes[0, 1].set_yticks(ticks_press)
    axes[0, 1].set_yticklabels([])

    
    
#     axes[2, 0].set_xticks(ticks_lat)
#     axes[2, 0].set_xticklabels([])

    fig.delaxes(axes[2,1])
        
        
    if save:
        save_path = '/glade/work/janniy/data_for_offline_momentum_paper/plots/'
        plt.savefig(save_path+ save_name +'.eps', format='eps', bbox_inches="tight")
        
#         from matplotlib.ticker import PercentFormatter

# #         data = [1000, 1000, 5000, 3000, 4000, 16000, 2000]

# #         axes[3,0].hist(np.reshape(field1,-1), weights=np.ones(np.reshape(field1,-1).shape) / np.reshape(field1,-1).shape,bins = 1000)
# #         axes[3,1].hist(np.reshape(field2,-1), weights=np.ones(np.reshape(field2,-1).shape) / np.reshape(field2,-1).shape,bins = 1000)

# #         axes[3,0].gca().yaxis.set_major_formatter(PercentFormatter(1))
    

In [1]:
def plot_mean_4_fields_production(Field,Field2,Field3,Field4,y,indices,p,save=0,save_name = 'del',with_trop=False,trop_p =np.zeros(180),
                                 plot_shear = False, shear = [],colorbar_costum_scale = False, 
                                  color1 = [], color1_ticks =[], color2 =[],color2_ticks =[],
                                  color3 = [], color3_ticks =[], color4 =[],color4_ticks =[],
                                  label1 = r'[kg m$^{-1}$ s$^{-2}$]', label3 = r'[m s$^{-2}$]', label2 = r'[kg m$^{-1}$ s$^{-2}$]', label4 = r'[m s$^{-2}$]',
#                                   label1 = r'[kg m$^{-1}$ s$^{-2}$]', label2 = r'[m s$^{-2}$]', label3 = r'[kg m$^{-1}$ s$^{-2}$]', label4 = r'[m s$^{-2}$]',
                                 headers = [r"Zonal momentum subgrid flux",r"Zonal momentum subgrid tendency",r"Meridional momentum subgrid flux",r"Meridional momentum subgrid tendency",r"Moisture surface subgrid flux",r"diffusivity RF"]):
    
    letters = ["(a)","(b)","(c)","(d)","(e)",'(f)']
#     headers = [r"Zonal momentum subgrid flux",r"Zonal momentum subgrid tendency",r"Meridional momentum subgrid flux",r"Meridional momentum subgrid tendency",r"Moisture surface subgrid flux",r"diffusivity RF"]

    y_plot = (y - np.mean(y))/10**6
    ticks_lat = [-5,0,5]
    
    ticks_press_tkz = [1000,800,600]
#     ticks_press = [900,600,300]
    ticks_press = [800,400,50]

    if colorbar_costum_scale:
        lev_Rsq = color1
        lev_Rsq_ticks = color1_ticks
        lev_Rsq3 = color3
        lev_Rsq_ticks3 = color3_ticks
    else:
        lev_Rsq = np.arange(-0.04, 0.041, 0.002) 
        lev_Rsq_ticks = np.arange(-0.03, 0.031, 0.02) 
        lev_Rsq3 = lev_Rsq
        lev_Rsq_ticks3 = lev_Rsq
    if colorbar_costum_scale:
        lev_Rsq2 = color2
        lev_Rsq_ticks2 = color2_ticks
        lev_Rsq4 = color4
        lev_Rsq_ticks4 = color4_ticks
    else:
        lev_Rsq2 = np.arange(-0.00004, 0.000041, 0.000002) 
        lev_Rsq_ticks2 = np.arange(-0.00003, 0.000031, 0.00002) 
        lev_Rsq4 = lev_Rsq2
        lev_Rsq_ticks4 = lev_Rsq2

#     ticks_press_R2 = [-0.006,-0.003, 0,0.003,0.006] #np.arange(-0.0075, 0.00751, 0.005)
    
    cont_min_rsq = 0
    cont_max_rsq = 1
    
    fontsize = 14
    fontsize_legend = 8
    h_width = 0.1
    v_width = 0.1
    h_width_cbar = 0.02
    colorbar_width =0.02

    p_lev_diff = 15
    
    row_dim = 3
    col_dim = 2

    fig_witdth = 0.7
    fig_height = 0.5
    fig_h_dist = 0.27
    fig_v_dist = 0.18
    
    size1 = 2
    fig, axes = plt.subplots(row_dim,col_dim)
    # fig.set_figheight(10)
    # fig.set_figwidth(10)
    upper_level_p = 48
    lowest_level = 0
    cmap1 = 'bwr'
    n=30
    yind1 = 65
    x_ind1 = 10
    # wnumber = 1
    ind_ind = 0
    
    
    x0 = axes[0,0].get_position().x0
    x1 = axes[0,0].get_position().x1
    y0 = axes[0,0].get_position().y0
    y1 = axes[0,0].get_position().y1

    for i in range(row_dim):
        for j  in range(col_dim):
            if i == 0 and j == 1:
                axes[i,j].set_position([x0 + j*fig_witdth + j*fig_h_dist,y0- i*fig_height - i*fig_v_dist,fig_witdth/1.0,fig_height])
            else:    
                axes[i,j].set_position([x0 + j*fig_witdth + j*fig_h_dist,y0- i*fig_height - i*fig_v_dist,fig_witdth,fig_height])
    
        
    levels = lev_Rsq
    i = 0
    print(Field.T.shape)
    print(p[0:Field.shape[1]].shape)
    print(y_plot.shape)
    cax_xy1 =axes[i,0].contourf(y_plot,p[0:Field.shape[1]],Field.T,levels = levels,cmap=cmap1, extend="both")
    axes[i, 0].set_ylim(np.amax(p[:]), np.amin(p[0:Field.shape[1]]))
    pos1_rsq = axes[i, 0].get_position() # get the original position 
    cbaxes_rsq = fig.add_axes([pos1_rsq.x0+pos1_rsq.width + h_width_cbar, pos1_rsq.y0, colorbar_width, fig_height]) 
#     cbar_rsq = fig.colorbar(cax_xy1,cax = cbaxes_rsq, ax=axes[i,0], shrink=1.0,ticks=np.arange(-0.1, 0.4, 0.1))
    cbar_rsq = fig.colorbar(cax_xy1,cax = cbaxes_rsq, ax=axes[i,0], shrink=1.0,ticks=lev_Rsq_ticks)
#     cbar_rsq.ax.set_xlabel(r'kg m$^{-2}$ s$^{-1}$', fontsize=fontsize)
    cbar_rsq.ax.tick_params(labelsize=fontsize-2)
#     cbar_rsq.set_label(r'[kg m$^{-1}$ s$^{-2}$]', rotation=90,fontsize=fontsize-2)
    cbar_rsq.set_label(label1, rotation=90,fontsize=fontsize-2)
    
    if plot_shear:
#         lev_shear_cont =np.arange(-0.026,0.026,0.002)# [0,100,200,300,400,500,600,700]
        lev_shear_cont = [-0.01,-0.003,0.003,0.01]
        cax_q_Rsq = axes[i, 0].contour(y_plot, p[0:shear.shape[0]], shear,lev_shear_cont,colors='k')#,vmin=cont_min_rsq, vmax=cont_max_rsq)#,cmap='seismic')
        axes[i, 0].set_ylim(np.amax(p), np.amin(p))
        # axes[1,0].clabel(cax_q_Rsq,cax_q_Rsq.levels[::6],fmt = '%1.0f', inline=1, fontsize=fontsize_cont1)
#         axes[i,0].clabel(cax_q_Rsq,lev_shear_cont[13],fmt = '%1.0f', inline=1, fontsize=fontsize-2)
        lev_shear_cont = [0]
        cax_q_Rsq = axes[i, 0].contour(y_plot, p[0:shear.shape[0]], shear,lev_shear_cont,colors='k')#,vmin=cont_min_rsq, vmax=cont_max_rsq)#,cmap='seismic')
        axes[i, 0].set_ylim(np.amax(p), np.amin(p))
        cax_q_Rsq.collections[0].set_linewidth(2) 

    
    levels = lev_Rsq3
    i = 1
    cax_xy1 =axes[i,0].contourf(y_plot,p[0:Field2.shape[1]],Field2.T,levels = levels,cmap=cmap1, extend="both")
    axes[i, 0].set_ylim(np.amax(p[:]), np.amin(p[0:Field2.shape[1]]))
    pos1_rsq = axes[i, 0].get_position() # get the original position 
    cbaxes_rsq = fig.add_axes([pos1_rsq.x0+pos1_rsq.width + h_width_cbar, pos1_rsq.y0, colorbar_width, fig_height]) 
#     cbar_rsq = fig.colorbar(cax_xy1,cax = cbaxes_rsq, ax=axes[i,0], shrink=1.0,ticks=np.arange(-0.1, 0.4, 0.1))
    cbar_rsq = fig.colorbar(cax_xy1,cax = cbaxes_rsq, ax=axes[i,0], shrink=1.0,ticks=lev_Rsq_ticks3)
#     cbar_rsq.ax.set_xlabel(r'kg m$^{-2}$ s$^{-1}$', fontsize=fontsize)
    cbar_rsq.ax.tick_params(labelsize=fontsize-2)
    cbar_rsq.set_label(label3, rotation=90,fontsize=fontsize-2)
    
    levels = lev_Rsq2
    i = 1
    cax_xy1 =axes[0,i].contourf(y_plot,p[0:Field3.shape[1]],Field3.T,levels = levels,cmap=cmap1, extend="both")
    axes[0, i].set_ylim(np.amax(p[:]), np.amin(p[0:Field3.shape[1]]))
    pos1_rsq = axes[0, i].get_position() # get the original position 
    cbaxes_rsq = fig.add_axes([pos1_rsq.x0+pos1_rsq.width + h_width_cbar, pos1_rsq.y0, colorbar_width, fig_height]) 
#     cbar_rsq = fig.colorbar(cax_xy1,cax = cbaxes_rsq, ax=axes[i,0], shrink=1.0,ticks=np.arange(-0.1, 0.4, 0.1))
    cbar_rsq = fig.colorbar(cax_xy1,cax = cbaxes_rsq, ax=axes[0,i], shrink=1.0,ticks=lev_Rsq_ticks2)
#     cbar_rsq.ax.set_xlabel(r'm s$^{-2}$', fontsize=fontsize)
    cbar_rsq.ax.tick_params(labelsize=fontsize-2)
    cbar_rsq.set_label(label2, rotation=90,fontsize=fontsize-2)
    
    levels = lev_Rsq4
    i = 1
    cax_xy1 =axes[1,i].contourf(y_plot,p[0:Field4.shape[1]],Field4.T,levels = levels,cmap=cmap1, extend="both")
    axes[1, i].set_ylim(np.amax(p[:]), np.amin(p[0:Field4.shape[1]]))
    pos1_rsq = axes[1, i].get_position() # get the original position 
    cbaxes_rsq = fig.add_axes([pos1_rsq.x0+pos1_rsq.width + h_width_cbar, pos1_rsq.y0, colorbar_width, fig_height]) 
#     cbar_rsq = fig.colorbar(cax_xy1,cax = cbaxes_rsq, ax=axes[i,0], shrink=1.0,ticks=np.arange(-0.1, 0.4, 0.1))
    cbar_rsq = fig.colorbar(cax_xy1,cax = cbaxes_rsq, ax=axes[1,i], shrink=1.0,ticks=lev_Rsq_ticks4)
#     cbar_rsq.ax.set_xlabel(r'm s$^{-2}$', fontsize=fontsize)
    cbar_rsq.ax.tick_params(labelsize=fontsize-2)
    cbar_rsq.set_label(label4, rotation=90,fontsize=fontsize-2)
    
    if with_trop:
        axes[0,0].plot(y_plot,trop_p,':k')
        axes[1,0].plot(y_plot,trop_p,':k')
        axes[0,1].plot(y_plot,trop_p,':k')
        axes[1,1].plot(y_plot,trop_p,':k')
    
    ind_let = 0
    for i in range(row_dim):
        for j  in range(col_dim):
            atext2 = axes[i, j].text(axes[0,0].get_position().x0*0.9+ j*fig_witdth + j*fig_h_dist,axes[0,0].get_position().y0*0.76- (i-1)*fig_height - (i-1)*fig_v_dist ,letters[ind_let],transform = fig.transFigure,color="black",fontsize=fontsize)

            header= axes[i, j].text(axes[0,0].get_position().x0*1.3+ j*fig_witdth + j*fig_h_dist,axes[0,0].get_position().y0*0.76- (i-1)*fig_height - (i-1)*fig_v_dist ,headers[ind_let],transform = fig.transFigure,color="black",fontsize=fontsize)
            ind_let = ind_let + 1

            
            
            
    axes[0, 0].set_xlim(np.amin(y_plot), np.amax(y_plot))
    axes[1, 0].set_xlim(np.amin(y_plot), np.amax(y_plot))
    axes[2, 0].set_xlim(np.amin(y_plot), np.amax(y_plot))
    axes[0, 1].set_xlim(np.amin(y_plot), np.amax(y_plot))
    axes[1, 1].set_xlim(np.amin(y_plot), np.amax(y_plot))
    


    axes[0, 0].set_ylabel('Pressure [hPa]',fontsize=fontsize)
    axes[1, 0].set_ylabel('Pressure [hPa]',fontsize=fontsize)
#     axes[2, 0].set_ylabel(r'kg m$^{-2}$ s$^{-1}$',fontsize=fontsize)
#     axes[1, 1].set_ylabel(r'kg m$^{-2}$ s$^{-1}$',fontsize=fontsize)


# axes[2, 0].set_xlabel('y [$10^3$ km]',fontsize=fontsize)
# axes[1, 1].set_xlabel('y [$10^3$ km]',fontsize=fontsize)


    axes[1, 0].set_xlabel('Latitude',fontsize=fontsize)
    axes[1, 1].set_xlabel('Latitude',fontsize=fontsize)
    
    fontsize_ticks = fontsize -2  
    ticks_lat_degrees = ['$-45^\circ$','$0^\circ$','$45^\circ$']
#     axes[1, 1].set_yticks(ticks_press_R2)
#     axes[1, 1].set_yticklabels(ticks_press_R2,fontsize=fontsize_ticks)
#     axes[2, 0].set_yticks(ticks_press_R2)
#     axes[2, 0].set_yticklabels(ticks_press_R2,fontsize=fontsize_ticks)

#     axes[2, 0].set_xticks(ticks_lat)
#     axes[2, 0].set_xticklabels(ticks_lat_degrees,fontsize=fontsize_ticks)
#     axes[1, 1].set_xticks(ticks_lat)
#     axes[1, 1].set_xticklabels(ticks_lat_degrees,fontsize=fontsize_ticks)
     
    axes[0, 0].set_xticks(ticks_lat)
    axes[0, 0].set_xticklabels([])
    axes[1, 0].set_xticks(ticks_lat)
    axes[1, 0].set_xticklabels(ticks_lat_degrees,fontsize=fontsize_ticks)
    axes[0, 1].set_xticks(ticks_lat)
    axes[0, 1].set_xticklabels([])
    
    axes[0, 0].set_yticks(ticks_press)
    axes[0, 0].set_yticklabels(ticks_press,fontsize=fontsize_ticks)
    
    axes[1, 0].set_yticks(ticks_press)
    axes[1, 0].set_yticklabels(ticks_press,fontsize=fontsize_ticks)

    axes[0, 1].set_yticks(ticks_press)
    axes[0, 1].set_yticklabels([])
    axes[1, 1].set_yticks(ticks_press)
    axes[1, 1].set_yticklabels([])
  
    axes[1, 1].set_xticks(ticks_lat)
    axes[1, 1].set_xticklabels(ticks_lat_degrees,fontsize=fontsize_ticks)

    
    
#     axes[2, 0].set_xticks(ticks_lat)
#     axes[2, 0].set_xticklabels([])

    fig.delaxes(axes[2,1])
    fig.delaxes(axes[2,0])
        
        
    if save:
        save_path = '/glade/work/janniy/data_for_offline_momentum_paper/plots/'
        plt.savefig(save_path+ save_name +'.eps', format='eps', bbox_inches="tight")
        
#         from matplotlib.ticker import PercentFormatter

# #         data = [1000, 1000, 5000, 3000, 4000, 16000, 2000]

# #         axes[3,0].hist(np.reshape(field1,-1), weights=np.ones(np.reshape(field1,-1).shape) / np.reshape(field1,-1).shape,bins = 1000)
# #         axes[3,1].hist(np.reshape(field2,-1), weights=np.ones(np.reshape(field2,-1).shape) / np.reshape(field2,-1).shape,bins = 1000)

# #         axes[3,0].gca().yaxis.set_major_formatter(PercentFormatter(1))
    

NameError: name 'np' is not defined

In [42]:
def plot_std_4_fields_production(Field,Field2,Field3,Field4,y,indices,p,save=0,save_name = 'del',with_trop=False,trop_p =np.zeros(180),
                                 lev_Rsq = np.arange(0, 0.51, 0.02), lev_Rsq_ticks = np.arange(0, 0.41, 0.2),
                                 lev_Rsq2 = np.arange(0, 0.00071, 0.00002),
                                 lev_Rsq_ticks2 = np.arange(0, 0.00061, 0.0002),
                                 headers = [r"Zonal momentum subgrid flux STD",r"Zonal momentum subgrid tendency STD",r"Meridional momentum subgrid flux STD",r"Meridional momentum subgrid tendency STD",r"Moisture surface subgrid flux",r"diffusivity RF"],
                                 legend_color1 = r'[kg m$^{-1}$ s$^{-2}$]',legend_color2 = r'[kg m$^{-1}$ s$^{-2}$]',legend_color3 = r'[m s$^{-2}$]',legend_color4 =r'[m s$^{-2}$]' ):
    
    letters = ["(a)","(b)","(c)","(d)","(e)",'(f)']
#     headers = [r"Zonal momentum subgrid flux STD",r"Zonal momentum subgrid tendency STD",r"Meridional momentum subgrid flux STD",r"Meridional momentum subgrid tendency STD",r"Moisture surface subgrid flux",r"diffusivity RF"]

    y_plot = (y - np.mean(y))/10**6
    ticks_lat = [-5,0,5]
    
    ticks_press_tkz = [1000,800,600]
    ticks_press = [900,600,300]

    
#     lev_Rsq = np.arange(0, 0.51, 0.02) 
#     lev_Rsq_ticks = np.arange(0, 0.41, 0.2) 

    
#     lev_Rsq = np.arange(0, 0.17, 0.02) 
#     lev_Rsq_ticks = np.arange(0, 0.11, 0.05) 

    
#     lev_Rsq2 = np.arange(0, 0.00071, 0.00002) 
#     lev_Rsq_ticks2 = np.arange(0, 0.00061, 0.0002) 

#     lev_Rsq2 = np.arange(0, 0.00031, 0.00002) 
#     lev_Rsq_ticks2 = np.arange(0, 0.00061, 0.0002) 


#     ticks_press_R2 = [-0.006,-0.003, 0,0.003,0.006] #np.arange(-0.0075, 0.00751, 0.005)
    
    cont_min_rsq = 0
    cont_max_rsq = 1
    
    fontsize = 14
    fontsize_legend = 8
    h_width = 0.1
    v_width = 0.1
    h_width_cbar = 0.02
    colorbar_width =0.02

    p_lev_diff = 15
    
    row_dim = 3
    col_dim = 2

    fig_witdth = 0.7
    fig_height = 0.5
    fig_h_dist = 0.27
    fig_v_dist = 0.18
    
    size1 = 2
    fig, axes = plt.subplots(row_dim,col_dim)
    # fig.set_figheight(10)
    # fig.set_figwidth(10)
    upper_level_p = 48
    lowest_level = 0
    cmap1 = 'YlGn'
    n=30
    yind1 = 65
    x_ind1 = 10
    # wnumber = 1
    ind_ind = 0
    
    
    x0 = axes[0,0].get_position().x0
    x1 = axes[0,0].get_position().x1
    y0 = axes[0,0].get_position().y0
    y1 = axes[0,0].get_position().y1

    for i in range(row_dim):
        for j  in range(col_dim):
            if i == 0 and j == 1:
                axes[i,j].set_position([x0 + j*fig_witdth + j*fig_h_dist,y0- i*fig_height - i*fig_v_dist,fig_witdth/1.0,fig_height])
            else:    
                axes[i,j].set_position([x0 + j*fig_witdth + j*fig_h_dist,y0- i*fig_height - i*fig_v_dist,fig_witdth,fig_height])
    
        
    levels = lev_Rsq
    i = 0
    cax_xy1 =axes[i,0].contourf(y_plot,p[0:Field.shape[1]],Field.T,levels = levels, extend="both")#,cmap=cmap1
    axes[i, 0].set_ylim(np.amax(p[:]), np.amin(p[0:Field.shape[1]]))
    pos1_rsq = axes[i, 0].get_position() # get the original position 
    cbaxes_rsq = fig.add_axes([pos1_rsq.x0+pos1_rsq.width + h_width_cbar, pos1_rsq.y0, colorbar_width, fig_height]) 
#     cbar_rsq = fig.colorbar(cax_xy1,cax = cbaxes_rsq, ax=axes[i,0], shrink=1.0,ticks=np.arange(-0.1, 0.4, 0.1))
    cbar_rsq = fig.colorbar(cax_xy1,cax = cbaxes_rsq, ax=axes[i,0], shrink=1.0,ticks=lev_Rsq_ticks)
#     cbar_rsq.ax.set_xlabel(r'kg m$^{-2}$ s$^{-1}$', fontsize=fontsize)
    cbar_rsq.ax.tick_params(labelsize=fontsize-2)
    cbar_rsq.set_label(legend_color1, rotation=90,fontsize=fontsize-2)
    
    levels = lev_Rsq
    i = 1
    cax_xy1 =axes[i,0].contourf(y_plot,p[0:Field2.shape[1]],Field2.T,levels = levels, extend="both")#,cmap=cmap1
    axes[i, 0].set_ylim(np.amax(p[:]), np.amin(p[0:Field2.shape[1]]))
    pos1_rsq = axes[i, 0].get_position() # get the original position 
    cbaxes_rsq = fig.add_axes([pos1_rsq.x0+pos1_rsq.width + h_width_cbar, pos1_rsq.y0, colorbar_width, fig_height]) 
#     cbar_rsq = fig.colorbar(cax_xy1,cax = cbaxes_rsq, ax=axes[i,0], shrink=1.0,ticks=np.arange(-0.1, 0.4, 0.1))
    cbar_rsq = fig.colorbar(cax_xy1,cax = cbaxes_rsq, ax=axes[i,0], shrink=1.0,ticks=lev_Rsq_ticks)
#     cbar_rsq.ax.set_xlabel(r'kg m$^{-2}$ s$^{-1}$', fontsize=fontsize)
    cbar_rsq.ax.tick_params(labelsize=fontsize-2)
    cbar_rsq.set_label(legend_color2, rotation=90,fontsize=fontsize-2)
    
    levels = lev_Rsq2
    i = 1
    cax_xy1 =axes[0,i].contourf(y_plot,p[0:Field3.shape[1]],Field3.T,levels = levels, extend="both")#cmap=cmap1,
    axes[0, i].set_ylim(np.amax(p[:]), np.amin(p[0:Field3.shape[1]]))
    pos1_rsq = axes[0, i].get_position() # get the original position 
    cbaxes_rsq = fig.add_axes([pos1_rsq.x0+pos1_rsq.width + h_width_cbar, pos1_rsq.y0, colorbar_width, fig_height]) 
#     cbar_rsq = fig.colorbar(cax_xy1,cax = cbaxes_rsq, ax=axes[i,0], shrink=1.0,ticks=np.arange(-0.1, 0.4, 0.1))
    cbar_rsq = fig.colorbar(cax_xy1,cax = cbaxes_rsq, ax=axes[0,i], shrink=1.0,ticks=lev_Rsq_ticks2)
#     cbar_rsq.ax.set_xlabel(r'm s$^{-2}$', fontsize=fontsize)
    cbar_rsq.ax.tick_params(labelsize=fontsize-2)
    cbar_rsq.set_label(legend_color3, rotation=90,fontsize=fontsize-2)
    
    levels = lev_Rsq2
    i = 1
    cax_xy1 =axes[1,i].contourf(y_plot,p[0:Field4.shape[1]],Field4.T,levels = levels, extend="both")#,cmap=cmap1
    axes[1, i].set_ylim(np.amax(p[:]), np.amin(p[0:Field4.shape[1]]))
    pos1_rsq = axes[1, i].get_position() # get the original position 
    cbaxes_rsq = fig.add_axes([pos1_rsq.x0+pos1_rsq.width + h_width_cbar, pos1_rsq.y0, colorbar_width, fig_height]) 
#     cbar_rsq = fig.colorbar(cax_xy1,cax = cbaxes_rsq, ax=axes[i,0], shrink=1.0,ticks=np.arange(-0.1, 0.4, 0.1))
    cbar_rsq = fig.colorbar(cax_xy1,cax = cbaxes_rsq, ax=axes[1,i], shrink=1.0,ticks=lev_Rsq_ticks2)
#     cbar_rsq.ax.set_xlabel(r'm s$^{-2}$', fontsize=fontsize)
    cbar_rsq.ax.tick_params(labelsize=fontsize-2)
    cbar_rsq.set_label(legend_color4, rotation=90,fontsize=fontsize-2)
    
    if with_trop:
        axes[0,0].plot(y_plot,trop_p,':k')
        axes[1,0].plot(y_plot,trop_p,':k')
        axes[0,1].plot(y_plot,trop_p,':k')
        axes[1,1].plot(y_plot,trop_p,':k')
    
    ind_let = 0
    for i in range(row_dim):
        for j  in range(col_dim):
            atext2 = axes[i, j].text(axes[0,0].get_position().x0*0.9+ j*fig_witdth + j*fig_h_dist,axes[0,0].get_position().y0*0.76- (i-1)*fig_height - (i-1)*fig_v_dist ,letters[ind_let],transform = fig.transFigure,color="black",fontsize=fontsize)

            header= axes[i, j].text(axes[0,0].get_position().x0*1.3+ j*fig_witdth + j*fig_h_dist,axes[0,0].get_position().y0*0.76- (i-1)*fig_height - (i-1)*fig_v_dist ,headers[ind_let],transform = fig.transFigure,color="black",fontsize=fontsize)
            ind_let = ind_let + 1

            
            
            
    axes[0, 0].set_xlim(np.amin(y_plot), np.amax(y_plot))
    axes[1, 0].set_xlim(np.amin(y_plot), np.amax(y_plot))
    axes[2, 0].set_xlim(np.amin(y_plot), np.amax(y_plot))
    axes[0, 1].set_xlim(np.amin(y_plot), np.amax(y_plot))
    axes[1, 1].set_xlim(np.amin(y_plot), np.amax(y_plot))
    


    axes[0, 0].set_ylabel('Pressure [hPa]',fontsize=fontsize)
    axes[1, 0].set_ylabel('Pressure [hPa]',fontsize=fontsize)
#     axes[2, 0].set_ylabel(r'kg m$^{-2}$ s$^{-1}$',fontsize=fontsize)
#     axes[1, 1].set_ylabel(r'kg m$^{-2}$ s$^{-1}$',fontsize=fontsize)


# axes[2, 0].set_xlabel('y [$10^3$ km]',fontsize=fontsize)
# axes[1, 1].set_xlabel('y [$10^3$ km]',fontsize=fontsize)


    axes[1, 0].set_xlabel('Latitude',fontsize=fontsize)
    axes[1, 1].set_xlabel('Latitude',fontsize=fontsize)
    
    fontsize_ticks = fontsize -2  
    ticks_lat_degrees = ['$-45^\circ$','$0^\circ$','$45^\circ$']
#     axes[1, 1].set_yticks(ticks_press_R2)
#     axes[1, 1].set_yticklabels(ticks_press_R2,fontsize=fontsize_ticks)
#     axes[2, 0].set_yticks(ticks_press_R2)
#     axes[2, 0].set_yticklabels(ticks_press_R2,fontsize=fontsize_ticks)

#     axes[2, 0].set_xticks(ticks_lat)
#     axes[2, 0].set_xticklabels(ticks_lat_degrees,fontsize=fontsize_ticks)
#     axes[1, 1].set_xticks(ticks_lat)
#     axes[1, 1].set_xticklabels(ticks_lat_degrees,fontsize=fontsize_ticks)
     
    axes[0, 0].set_xticks(ticks_lat)
    axes[0, 0].set_xticklabels([])
    axes[1, 0].set_xticks(ticks_lat)
    axes[1, 0].set_xticklabels(ticks_lat_degrees,fontsize=fontsize_ticks)
    axes[0, 1].set_xticks(ticks_lat)
    axes[0, 1].set_xticklabels([])
    
    axes[0, 0].set_yticks(ticks_press)
    axes[0, 0].set_yticklabels(ticks_press,fontsize=fontsize_ticks)
    
    axes[1, 0].set_yticks(ticks_press)
    axes[1, 0].set_yticklabels(ticks_press,fontsize=fontsize_ticks)

    axes[0, 1].set_yticks(ticks_press)
    axes[0, 1].set_yticklabels([])
    axes[1, 1].set_yticks(ticks_press)
    axes[1, 1].set_yticklabels([])
  
    axes[1, 1].set_xticks(ticks_lat)
    axes[1, 1].set_xticklabels(ticks_lat_degrees,fontsize=fontsize_ticks)

    
    
#     axes[2, 0].set_xticks(ticks_lat)
#     axes[2, 0].set_xticklabels([])

    fig.delaxes(axes[2,1])
    fig.delaxes(axes[2,0])
        
        
    if save:
        save_path = '/glade/work/janniy/data_for_offline_momentum_paper/plots/'
        plt.savefig(save_path+ save_name +'.eps', format='eps', bbox_inches="tight")
        
#         from matplotlib.ticker import PercentFormatter

# #         data = [1000, 1000, 5000, 3000, 4000, 16000, 2000]

# #         axes[3,0].hist(np.reshape(field1,-1), weights=np.ones(np.reshape(field1,-1).shape) / np.reshape(field1,-1).shape,bins = 1000)
# #         axes[3,1].hist(np.reshape(field2,-1), weights=np.ones(np.reshape(field2,-1).shape) / np.reshape(field2,-1).shape,bins = 1000)

# #         axes[3,0].gca().yaxis.set_major_formatter(PercentFormatter(1))
    

## Compare to Seager parameterization. 

In [68]:
def calc_precip(q, rho, z):
# surface precipitation rate given tendency of specific humidity
    precip = -vertical_integral(q, rho, z)
    return precip

def vertical_integral(data, rho, z):
# vertical integral with respect to sigma
    rho_dz = vertical_diff(rho, z)
    int_data = np.sum(data * rho_dz[:,None], axis=0)
    return int_data

def vertical_diff(rho, z):
    # follow vertical differencing from setgrid.f90 in SAM
    # changed indexing from starting at 1 to 0
    nzm = z.size
    adz = np.zeros(nzm)    
    dz = 0.5*(z[0]+z[1]) 
    adz[0] = 1.
    for k in range(1,nzm-1): # range doesn't include stopping number
        adz[k] = 0.5*(z[k+1]-z[k-1])/dz

    adz[nzm-1] = (z[nzm-1]-z[nzm-2])/dz

    rho_dz = adz*dz*rho

    return rho_dz

In [1]:
def plot_mean_4_2Dfields_2_1D_production(Field,Field2,Field3,Field4,y,indices,p, precip_mean_trop,
                                         seager_param, true_out_xt_avg, pred_out_xt_avg, pearson_lev = 11200,save=0,
                                         save_name = 'del',with_trop=False,trop_p =np.zeros(180),
                                 plot_shear = False, shear = [],colorbar_costum_scale = False, 
                                  color1 = [], color1_ticks =[], color2 =[],color2_ticks =[],
                                  color3 = [], color3_ticks =[], color4 =[],color4_ticks =[],
                                  label1 = r'[kg m$^{-1}$ s$^{-2}$]', label2 = r'[m s$^{-2}$]', label3 = r'[kg m$^{-1}$ s$^{-2}$]', label4 = r'[m s$^{-2}$]',
                                    headers = [r"Zonal momentum subgrid flux",r"Zonal momentum subgrid tendency",r"Meridional momentum subgrid flux",r"Meridional momentum subgrid tendency",r"Moisture surface subgrid flux",r"diffusivity RF"]
                                 ):
    
    letters = ["(a)","(b)","(c)","(d)","(e)",'(f)']
#     headers = [r"Zonal momentum subgrid flux",r"Zonal momentum subgrid tendency",r"Meridional momentum subgrid flux",r"Meridional momentum subgrid tendency",r"Moisture surface subgrid flux",r"diffusivity RF"]

    y_plot = (y - np.mean(y))/10**6
    ticks_lat = [-5,0,5]
    
    ticks_press_tkz = [1000,800,600]
#     ticks_press = [900,600,300]
    ticks_press = [800,400,50]

    if colorbar_costum_scale:
        lev_Rsq = color1
        lev_Rsq_ticks = color1_ticks
        lev_Rsq3 = color3
        lev_Rsq_ticks3 = color3_ticks
    else:
        lev_Rsq = np.arange(-0.04, 0.041, 0.002) 
        lev_Rsq_ticks = np.arange(-0.03, 0.031, 0.02) 
        lev_Rsq3 = lev_Rsq
        lev_Rsq_ticks3 = lev_Rsq
    if colorbar_costum_scale:
        lev_Rsq2 = color2
        lev_Rsq_ticks2 = color2_ticks
        lev_Rsq4 = color4
        lev_Rsq_ticks4 = color4_ticks
    else:
        lev_Rsq2 = np.arange(-0.00004, 0.000041, 0.000002) 
        lev_Rsq_ticks2 = np.arange(-0.00003, 0.000031, 0.00002) 
        lev_Rsq4 = lev_Rsq2
        lev_Rsq_ticks4 = lev_Rsq2

#     ticks_press_R2 = [-0.006,-0.003, 0,0.003,0.006] #np.arange(-0.0075, 0.00751, 0.005)
    
    cont_min_rsq = 0
    cont_max_rsq = 1
    
    fontsize = 14
    fontsize_legend = 8
    h_width = 0.1
    v_width = 0.1
    h_width_cbar = 0.02
    colorbar_width =0.02

    p_lev_diff = 15
    
    row_dim = 3
    col_dim = 2

    fig_witdth = 0.7
    fig_height = 0.5
    fig_h_dist = 0.27
    fig_v_dist = 0.22
    
    size1 = 2
    fig, axes = plt.subplots(row_dim,col_dim)
    # fig.set_figheight(10)
    # fig.set_figwidth(10)
    upper_level_p = 48
    lowest_level = 0
    cmap1 = 'bwr'
    n=30
    yind1 = 65
    x_ind1 = 10
    # wnumber = 1
    ind_ind = 0
    
    
    x0 = axes[0,0].get_position().x0
    x1 = axes[0,0].get_position().x1
    y0 = axes[0,0].get_position().y0
    y1 = axes[0,0].get_position().y1

    for i in range(row_dim):
        for j  in range(col_dim):
            if i == 0 and j == 1:
                axes[i,j].set_position([x0 + j*fig_witdth + j*fig_h_dist,y0- i*fig_height - i*fig_v_dist,fig_witdth/1.0,fig_height])
            else:    
                axes[i,j].set_position([x0 + j*fig_witdth + j*fig_h_dist,y0- i*fig_height - i*fig_v_dist,fig_witdth,fig_height])
    
        
    levels = lev_Rsq
    i = 0
    print(Field.T.shape)
    print(p[0:Field.shape[1]].shape)
    print(y_plot.shape)
    cax_xy1 =axes[i,0].contourf(y_plot,p[0:Field.shape[1]],Field.T,levels = levels,cmap=cmap1, extend="both")
    axes[i, 0].set_ylim(np.amax(p[:]), np.amin(p[0:Field.shape[1]]))
    pos1_rsq = axes[i, 0].get_position() # get the original position 
    cbaxes_rsq = fig.add_axes([pos1_rsq.x0+pos1_rsq.width + h_width_cbar, pos1_rsq.y0, colorbar_width, fig_height]) 
#     cbar_rsq = fig.colorbar(cax_xy1,cax = cbaxes_rsq, ax=axes[i,0], shrink=1.0,ticks=np.arange(-0.1, 0.4, 0.1))
    cbar_rsq = fig.colorbar(cax_xy1,cax = cbaxes_rsq, ax=axes[i,0], shrink=1.0,ticks=lev_Rsq_ticks)
#     cbar_rsq.ax.set_xlabel(r'kg m$^{-2}$ s$^{-1}$', fontsize=fontsize)
    cbar_rsq.ax.tick_params(labelsize=fontsize-2)
#     cbar_rsq.set_label(r'[kg m$^{-1}$ s$^{-2}$]', rotation=90,fontsize=fontsize-2)
    cbar_rsq.set_label(label1, rotation=90,fontsize=fontsize-2)
    
    if plot_shear:
#         lev_shear_cont =np.arange(-0.026,0.026,0.002)# [0,100,200,300,400,500,600,700]
        lev_shear_cont = [-0.01,-0.003,0.003,0.01]
        cax_q_Rsq = axes[i, 0].contour(y_plot, p[0:shear.shape[0]], shear,lev_shear_cont,colors='k')#,vmin=cont_min_rsq, vmax=cont_max_rsq)#,cmap='seismic')
        axes[i, 0].set_ylim(np.amax(p), np.amin(p))
        # axes[1,0].clabel(cax_q_Rsq,cax_q_Rsq.levels[::6],fmt = '%1.0f', inline=1, fontsize=fontsize_cont1)
#         axes[i,0].clabel(cax_q_Rsq,lev_shear_cont[13],fmt = '%1.0f', inline=1, fontsize=fontsize-2)
        lev_shear_cont = [0]
        cax_q_Rsq = axes[i, 0].contour(y_plot, p[0:shear.shape[0]], shear,lev_shear_cont,colors='k')#,vmin=cont_min_rsq, vmax=cont_max_rsq)#,cmap='seismic')
        axes[i, 0].set_ylim(np.amax(p), np.amin(p))
        cax_q_Rsq.collections[0].set_linewidth(2) 

    
    levels = lev_Rsq3
    i = 1
    cax_xy1 =axes[i,0].contourf(y_plot,p[0:Field2.shape[1]],Field2.T,levels = levels,cmap=cmap1, extend="both")
    axes[i, 0].set_ylim(np.amax(p[:]), np.amin(p[0:Field2.shape[1]]))
    pos1_rsq = axes[i, 0].get_position() # get the original position 
    cbaxes_rsq = fig.add_axes([pos1_rsq.x0+pos1_rsq.width + h_width_cbar, pos1_rsq.y0, colorbar_width, fig_height]) 
#     cbar_rsq = fig.colorbar(cax_xy1,cax = cbaxes_rsq, ax=axes[i,0], shrink=1.0,ticks=np.arange(-0.1, 0.4, 0.1))
    cbar_rsq = fig.colorbar(cax_xy1,cax = cbaxes_rsq, ax=axes[i,0], shrink=1.0,ticks=lev_Rsq_ticks3)
#     cbar_rsq.ax.set_xlabel(r'kg m$^{-2}$ s$^{-1}$', fontsize=fontsize)
    cbar_rsq.ax.tick_params(labelsize=fontsize-2)
    cbar_rsq.set_label(label3, rotation=90,fontsize=fontsize-2)
    
    levels = lev_Rsq2
    i = 1
    cax_xy1 =axes[0,i].contourf(y_plot,p[0:Field3.shape[1]],Field3.T,levels = levels,cmap=cmap1, extend="both")
    axes[0, i].set_ylim(np.amax(p[:]), np.amin(p[0:Field3.shape[1]]))
    pos1_rsq = axes[0, i].get_position() # get the original position 
    cbaxes_rsq = fig.add_axes([pos1_rsq.x0+pos1_rsq.width + h_width_cbar, pos1_rsq.y0, colorbar_width, fig_height]) 
#     cbar_rsq = fig.colorbar(cax_xy1,cax = cbaxes_rsq, ax=axes[i,0], shrink=1.0,ticks=np.arange(-0.1, 0.4, 0.1))
    cbar_rsq = fig.colorbar(cax_xy1,cax = cbaxes_rsq, ax=axes[0,i], shrink=1.0,ticks=lev_Rsq_ticks2)
#     cbar_rsq.ax.set_xlabel(r'm s$^{-2}$', fontsize=fontsize)
    cbar_rsq.ax.tick_params(labelsize=fontsize-2)
    cbar_rsq.set_label(label2, rotation=90,fontsize=fontsize-2)
    
    levels = lev_Rsq4
    i = 1
    cax_xy1 =axes[1,i].contourf(y_plot,p[0:Field4.shape[1]],Field4.T,levels = levels,cmap=cmap1, extend="both")
    axes[1, i].set_ylim(np.amax(p[:]), np.amin(p[0:Field4.shape[1]]))
    pos1_rsq = axes[1, i].get_position() # get the original position 
    cbaxes_rsq = fig.add_axes([pos1_rsq.x0+pos1_rsq.width + h_width_cbar, pos1_rsq.y0, colorbar_width, fig_height]) 
#     cbar_rsq = fig.colorbar(cax_xy1,cax = cbaxes_rsq, ax=axes[i,0], shrink=1.0,ticks=np.arange(-0.1, 0.4, 0.1))
    cbar_rsq = fig.colorbar(cax_xy1,cax = cbaxes_rsq, ax=axes[1,i], shrink=1.0,ticks=lev_Rsq_ticks4)
#     cbar_rsq.ax.set_xlabel(r'm s$^{-2}$', fontsize=fontsize)
    cbar_rsq.ax.tick_params(labelsize=fontsize-2)
    cbar_rsq.set_label(label4, rotation=90,fontsize=fontsize-2)
    
    if with_trop:
        axes[0,0].plot(y_plot,trop_p,':k')
        axes[1,0].plot(y_plot,trop_p,':k')
        axes[0,1].plot(y_plot,trop_p,':k')
        axes[1,1].plot(y_plot,trop_p,':k')
    
    ind_let = 0
    for i in range(row_dim):
        for j  in range(col_dim):
            atext2 = axes[i, j].text(axes[0,0].get_position().x0*0.9+ j*fig_witdth + j*fig_h_dist,axes[0,0].get_position().y0*0.7- (i-1)*fig_height - (i-1)*fig_v_dist ,letters[ind_let],transform = fig.transFigure,color="black",fontsize=fontsize)

            header= axes[i, j].text(axes[0,0].get_position().x0*1.3+ j*fig_witdth + j*fig_h_dist,axes[0,0].get_position().y0*0.7- (i-1)*fig_height - (i-1)*fig_v_dist ,headers[ind_let],transform = fig.transFigure,color="black",fontsize=fontsize)
            ind_let = ind_let + 1

            
            
            
    axes[0, 0].set_xlim(np.amin(y_plot), np.amax(y_plot))
    axes[1, 0].set_xlim(np.amin(y_plot), np.amax(y_plot))
    axes[2, 0].set_xlim(np.amin(y_plot), np.amax(y_plot))
    axes[0, 1].set_xlim(np.amin(y_plot), np.amax(y_plot))
    axes[1, 1].set_xlim(np.amin(y_plot), np.amax(y_plot))
    


    axes[0, 0].set_ylabel('Pressure [hPa]',fontsize=fontsize)
    axes[1, 0].set_ylabel('Pressure [hPa]',fontsize=fontsize)
#     axes[2, 0].set_ylabel(r'kg m$^{-2}$ s$^{-1}$',fontsize=fontsize)
#     axes[1, 1].set_ylabel(r'kg m$^{-2}$ s$^{-1}$',fontsize=fontsize)


# axes[2, 0].set_xlabel('y [$10^3$ km]',fontsize=fontsize)
# axes[1, 1].set_xlabel('y [$10^3$ km]',fontsize=fontsize)


    axes[1, 0].set_xlabel('Latitude',fontsize=fontsize)
    axes[1, 1].set_xlabel('Latitude',fontsize=fontsize)
    
    fontsize_ticks = fontsize -2  
    ticks_lat_degrees = ['$-45^\circ$','$0^\circ$','$45^\circ$']
#     axes[1, 1].set_yticks(ticks_press_R2)
#     axes[1, 1].set_yticklabels(ticks_press_R2,fontsize=fontsize_ticks)
#     axes[2, 0].set_yticks(ticks_press_R2)
#     axes[2, 0].set_yticklabels(ticks_press_R2,fontsize=fontsize_ticks)

#     axes[2, 0].set_xticks(ticks_lat)
#     axes[2, 0].set_xticklabels(ticks_lat_degrees,fontsize=fontsize_ticks)
#     axes[1, 1].set_xticks(ticks_lat)
#     axes[1, 1].set_xticklabels(ticks_lat_degrees,fontsize=fontsize_ticks)
     
    axes[0, 0].set_xticks(ticks_lat)
    axes[0, 0].set_xticklabels([])
    axes[1, 0].set_xticks(ticks_lat)
    axes[1, 0].set_xticklabels(ticks_lat_degrees,fontsize=fontsize_ticks)
    axes[0, 1].set_xticks(ticks_lat)
    axes[0, 1].set_xticklabels([])
    
    axes[0, 0].set_yticks(ticks_press)
    axes[0, 0].set_yticklabels(ticks_press,fontsize=fontsize_ticks)
    
    axes[1, 0].set_yticks(ticks_press)
    axes[1, 0].set_yticklabels(ticks_press,fontsize=fontsize_ticks)

    axes[0, 1].set_yticks(ticks_press)
    axes[0, 1].set_yticklabels([])
    axes[1, 1].set_yticks(ticks_press)
    axes[1, 1].set_yticklabels([])
  
    axes[1, 1].set_xticks(ticks_lat)
    axes[1, 1].set_xticklabels(ticks_lat_degrees,fontsize=fontsize_ticks)

    
    
#     axes[2, 0].set_xticks(ticks_lat)
#     axes[2, 0].set_xticklabels([])

#     fig.delaxes(axes[2,1])
#     fig.delaxes(axes[2,0])
  
    true_range1 = -0.0000028
    true_range2 = 0.0000075

    true_ticks = [0,0.000003,0.000006]

    mm_rain = 0
    corr_paramet, _ = pearsonr(seager_param[precip_mean_trop>2],true_out_xt_avg[precip_mean_trop>2])
    corr_paramet_RF, _ = pearsonr(true_out_xt_avg[precip_mean_trop>2],pred_out_xt_avg[precip_mean_trop>2])
    slope, intercept, r_value, p_value, std_err = stats.linregress(seager_param[precip_mean_trop>2],true_out_xt_avg[precip_mean_trop>2])

    slope_no_intercept, _, _, _ = np.linalg.lstsq(seager_param[precip_mean_trop>2,None],true_out_xt_avg[precip_mean_trop>2]) 
    # = stats.linregress(seager_param[precip_mean_trop>2],true_out_xt_avg[precip_mean_trop>2])


    marker1 = 100
    markers_style = ['x','^','o']

    colors1 = ['red','black','blue','green']
    cax_mean_online2 =axes[2,0].scatter(seager_param[precip_mean_trop>2],true_out_xt_avg[precip_mean_trop>2])
    axes[2,0].set_xlim(-75, 50)
    axes[2,0].set_ylim(true_range1, true_range2)
    cax_mean_online3 = axes[2,1].scatter(pred_out_xt_avg[precip_mean_trop>2],true_out_xt_avg[precip_mean_trop>2])
    axes[2,1].set_xlim(true_range1, true_range2)
    axes[2,1].set_ylim(true_range1, true_range2)

    axes[2, 1].set_xticks(true_ticks)
    axes[2, 1].set_xticklabels(true_ticks,fontsize=fontsize)
    axes[2, 1].set_yticks(true_ticks)
    axes[2, 1].set_yticklabels([])

    axes[2, 0].set_yticks(true_ticks)
    axes[2, 0].set_yticklabels(true_ticks,fontsize=fontsize)
    axes[2, 0].set_xticks([-80,-40,0,40])
    axes[2, 0].set_xticklabels([-80,-40,0,40],fontsize=fontsize)

    
    axes[2, 1].set_xlabel('NN prediction @ (' + "{:.1f}".format(pearson_lev/1000) + 'km)' + '[m s$^{-2}$]',fontsize=fontsize)
    axes[2, 0].set_xlabel(r'Prec*(u(0.6km)- u(' + "{:.1f}".format(pearson_lev/1000) + 'km)' ,fontsize=fontsize)
    axes[2, 0].set_ylabel(r'True u tend @ (' + "{:.1f}".format(pearson_lev/1000) + 'km)' + r'[m s$^{-2}$]',fontsize=fontsize)
    axes[2, 0].text(0, 0, 'Pearson:' + str(round(corr_paramet,3)),fontsize=fontsize)
    axes[2, 0].text(0, 0-0.0000014, 'slope:' + "{:.2e}".format(slope_no_intercept[0]),fontsize=fontsize)
    axes[2, 1].text(4e-6, 0, 'Pearson:' + str(round(corr_paramet_RF,3)),fontsize=fontsize)

    axes[2,0].spines["top"].set_visible(False)
    axes[2,0].spines["right"].set_visible(False)
    axes[2,1].spines["top"].set_visible(False)
    axes[2,1].spines["right"].set_visible(False)
        
    if save:
        save_path = '/glade/work/janniy/data_for_offline_momentum_paper/plots/'
        plt.savefig(save_path+ save_name +'.eps', format='eps', bbox_inches="tight")
        
#         from matplotlib.ticker import PercentFormatter

# #         data = [1000, 1000, 5000, 3000, 4000, 16000, 2000]

# #         axes[3,0].hist(np.reshape(field1,-1), weights=np.ones(np.reshape(field1,-1).shape) / np.reshape(field1,-1).shape,bins = 1000)
# #         axes[3,1].hist(np.reshape(field2,-1), weights=np.ones(np.reshape(field2,-1).shape) / np.reshape(field2,-1).shape,bins = 1000)

# #         axes[3,0].gca().yaxis.set_major_formatter(PercentFormatter(1))
    

NameError: name 'np' is not defined

In [74]:
def vertical_integral3D(data, rho, z):
# vertical integral with respect to sigma
    rho_dz = vertical_diff(rho, z)
    int_data = np.sum(data * rho_dz[:,None,None,None], axis=0)
    return int_data

In [89]:
def plot_sapshots_no_int_fluxes(true_field,pred_field,true_field_int_z,pred_field_int_z,
                  precip,p,y_i =80,change_range=0,save_flag = False,save_name1= 'del_snap',
                  plot_trop=False,trop_p = [],
                  x_ind1 =10, time_ind1=-100, time_ind2 = -1,
                  cosutm_ticks=False,
                  tick1=[]
                   ):
    per_day = 3600*24
    fontsize = 15
    fontsize_legend = 8
    h_width = 0.1
    v_width = 0.1
    h_width_cbar = 0.02
    colorbar_width =0.02

    row_dim = 3
    col_dim = 2

    fig_witdth = 1.
    fig_height = 0.75
    fig_h_dist = 0.2
    fig_v_dist = 0.2
    
    if cosutm_ticks:
        flux_ticks_colorbar = tick1
    else:
        flux_ticks_colorbar = np.arange(-0.6,0.65,0.2)

    letters = ["(a) True flux","(b) y_i = 43","(b) Predicted flux","(d)",'(c) Integrated fluxes and precipitation','','','','','']

    size1 = 2
    fig, axes = plt.subplots(row_dim,col_dim)
    # fig.set_figheight(10)
    # fig.set_figwidth(10)
    upper_level_p = 47
    lowest_level = 0
    cmap1 = 'bwr'
    time_steps = 100
    time = np.arange(0,time_steps*3,3)
    n=31
    yind1 = y_i
#     x_ind1 = 10
    # wnumber = 1
    if cosutm_ticks:
        range11 = np.max(tick1)
    else:
        range11 = np.max(np.abs((true_u_flux[:,-time_steps:,yind1,x_ind1])))/2.5
    levels = np.linspace(-range11, range11, n+1)
    cax_mean_online2 =axes[0,0].contourf(time,p[lowest_level:upper_level_p],true_field[lowest_level:upper_level_p,time_ind1:time_ind2,yind1,x_ind1],levels = levels,cmap=cmap1, extend="both")
    axes[0, 0].set_ylim(np.amax(p[lowest_level]), np.amin(p))
    if plot_trop:
        axes[0, 0].plot(time,trop_p)

    if change_range == 1:
        range11 = np.max(np.abs((pred_field[:,-time_steps:,yind1,x_ind1])))/2.5
        levels = np.linspace(-range11, range11, n+1)
        upper_level_p = pred_field.shape[0]
    cax_mean_online3 =axes[1,0].contourf(time,p[lowest_level:upper_level_p],pred_field[lowest_level:upper_level_p,time_ind1:time_ind2,yind1,x_ind1],levels = levels,cmap=cmap1, extend="both")
    axes[1, 0].set_ylim(np.amax(p[lowest_level]), np.amin(p))
    if plot_trop:
        axes[1, 0].plot(time,trop_p)

    


    color = 'black'
    lns1 =axes[2,0].plot(time,precip[yind1,x_ind1,time_ind1:time_ind2], color=color,label='precipitation')
    axes[2, 0].set_xlim(0, time[-1])
    axes[2, 0].tick_params(axis='y', labelcolor=color)


#     ax2 = axes[2, 0].twinx()
#     color = 'tab:blue'
#     ax2.set_ylabel(r'Subgrid flux [kg m$^{-1}$ s$^{-2}$]', color=color,fontsize=fontsize)  # we already handled the x-label with ax1
#     lns2 =ax2.plot(time,true_field_int_z[time_ind1:time_ind2,yind1,x_ind1], color=color,label='integrated flux (true)')
#     ax2.tick_params(axis='y', labelcolor=color)
#     lns3 =ax2.plot(time,pred_field_int_z[time_ind1:time_ind2,yind1,x_ind1], 'r:',label='integrated flux (predicted)')
    
#     # added these three lines
#     lns = lns1+lns2+lns3
#     labs = [l.get_label() for l in lns]
#     axes[2, 0].legend(lns, labs, loc=0,frameon=False)
   


    x0 = axes[0,0].get_position().x0
    x1 = axes[0,0].get_position().x1
    y0 = axes[0,0].get_position().y0
    y1 = axes[0,0].get_position().y1

    for i in range(row_dim):
        for j  in range(col_dim):
            axes[i,j].set_position([x0 + j*fig_witdth + j*fig_h_dist,y0- i*fig_height - i*fig_v_dist,fig_witdth,fig_height])


    ind_let = 0
    for i in range(row_dim):
        for j  in range(col_dim):
            atext2 = axes[i, j].text(axes[0,0].get_position().x0*1.1+ j*fig_witdth + j*fig_h_dist,axes[0,0].get_position().y0*0.76- (i-1)*fig_height - (i-1)*fig_v_dist ,letters[ind_let],transform = fig.transFigure,color="black",fontsize=fontsize)

    #         header= axes[i, j].text(axes[0,0].get_position().x0*1.5+ j*fig_witdth + j*fig_h_dist,axes[0,0].get_position().y0*0.92- (i-1)*fig_height - (i-1)*fig_v_dist ,headers[ind_let],transform = fig.transFigure,color="black",fontsize=fontsize)
            ind_let = ind_let + 1

    pos1_rsq = axes[0, 0].get_position() # get the original position 
    cbaxes_rsq = fig.add_axes([pos1_rsq.x0+pos1_rsq.width + h_width_cbar, pos1_rsq.y0, colorbar_width, fig_height]) 
    cbar_rsq = fig.colorbar(cax_mean_online2,cax = cbaxes_rsq, ax=axes[0,0], shrink=1.0,ticks=flux_ticks_colorbar)
    cbar_rsq.ax.tick_params(labelsize=fontsize) 
    cbar_rsq.set_label(r'[kg m$^{-1}$ sec$^{-2}$]',fontsize=fontsize)
    
    if change_range == 1:
        flux_ticks_colorbar = np.arange(-range11,range11,range11/10)
    pos1_rsq = axes[1, 0].get_position() # get the original position 
    cbaxes_rsq = fig.add_axes([pos1_rsq.x0+pos1_rsq.width + h_width_cbar, pos1_rsq.y0, colorbar_width, fig_height]) 
#     cbar_rsq = fig.colorbar(cax_mean_online3,cax = cbaxes_rsq, ax=axes[1,0], shrink=1.0,ticks=flux_ticks_colorbar)
    cbar_rsq = fig.colorbar(cax_mean_online3,cax = cbaxes_rsq, ax=axes[1,0], shrink=1.0,ticks=flux_ticks_colorbar)
    cbar_rsq.ax.tick_params(labelsize=fontsize) 
    cbar_rsq.set_label(r'[kg m$^{-1}$ sec$^{-2}$]',fontsize=fontsize)




    axes[2, 0].set_xlabel('Time (hours)',fontsize=fontsize)
    axes[1, 1].set_xlabel('Time (hours)',fontsize=fontsize)
    axes[2, 0].set_ylabel(r'Precipitation  [mm day$^{-1}$]',fontsize=fontsize)
    axes[0, 0].set_ylabel('Pressure [hPa]',fontsize=fontsize)
    axes[1, 0].set_ylabel('Pressure [hPa]',fontsize=fontsize)


    # axes[0, 0].set_yticks(ticks_pmean)
    axes[0, 1].set_yticklabels([])
    # axes[1, 1].set_yticklabels([])


    
    axes[2,0].spines["top"].set_visible(False)
    axes[2,0].spines["right"].set_visible(False)

#     precip_ticks = [0,50,100,150,200,250,300,350,400,450]  
    axes[2, 0].set_ylim(0,200)
#     ax2.set_ylim(-7000,8000)
    precip_ticks_label = [0,50,100,150]#,200,250,300]
    precip_ticks = precip_ticks_label
    axes[2, 0].set_yticks(precip_ticks)
    axes[2, 0].set_yticklabels(precip_ticks_label,fontsize=fontsize)

#     flux_ticks = [-2000,0,2000,4000,6000,8000]
#     flux_ticks = [-6000,-3000,0,3000,6000]
#     flux_ticks_label = [0,3000,6000]
#     flux_ticks = flux_ticks_label
#     ax2.set_yticks(flux_ticks_label)
#     ax2.set_yticklabels(flux_ticks_label,fontsize=fontsize)

    pressure_ticks = [300,600,900]
    time_ticks = [0,120,240]
    axes[0, 0].set_xticks(time_ticks)
    axes[0, 0].set_xticklabels([])
    axes[1, 0].set_xticks(time_ticks)
    axes[1, 0].set_xticklabels([])
    axes[2, 0].set_xticks(time_ticks)
    axes[2, 0].set_xticklabels(time_ticks,fontsize=fontsize)

    axes[0, 0].set_yticks(pressure_ticks)
    axes[0, 0].set_yticklabels(pressure_ticks,fontsize=fontsize)
    axes[1, 0].set_yticks(pressure_ticks)
    axes[1, 0].set_yticklabels(pressure_ticks,fontsize=fontsize)

    fig.delaxes(axes[0,1])
    fig.delaxes(axes[1,1])
    fig.delaxes(axes[2,1])

    save_path = '/glade/work/janniy/data_for_offline_momentum_paper/plots/'
    if save_flag:
        plt.savefig(save_path+save_name1 + '.eps', format='eps', bbox_inches="tight")
    
#     plt.savefig(save_path+'snapshot_u_flux_precip.eps', format='eps', bbox_inches="tight")
#     plt.savefig(save_path+'snapshot_u_flux_T_flux_show_noise2.eps', format='eps', bbox_inches="tight")
    # plt.savefig(save_path+'correcting_u_precip_subgrid_hovemoller_tropics2_int_flux.eps', format='eps', bbox_inches="tight")

In [90]:
def plot_sapshots(true_field,pred_field,true_field_int_z,pred_field_int_z,
                  precip,p,y_i =80,change_range=0,save_flag = False,save_name1= 'del_snap',
                  plot_trop=False,trop_p = [],
                  x_ind1 =10, time_ind1=-100, time_ind2 = -1
                 ):
    per_day = 3600*24
    fontsize = 15
    fontsize_legend = 8
    h_width = 0.1
    v_width = 0.1
    h_width_cbar = 0.02
    colorbar_width =0.02

    row_dim = 3
    col_dim = 2

    fig_witdth = 1.
    fig_height = 0.75
    fig_h_dist = 0.2
    fig_v_dist = 0.2
    
    flux_ticks_colorbar = np.arange(-0.6,0.65,0.2)

    letters = ["(a) True flux","(b) y_i = 43","(b) Predicted flux","(d)",'(c) Integrated fluxes and precipitation','','','','','']

    size1 = 2
    fig, axes = plt.subplots(row_dim,col_dim)
    # fig.set_figheight(10)
    # fig.set_figwidth(10)
    upper_level_p = 47
    lowest_level = 0
    cmap1 = 'bwr'
    time_steps = 100
    time = np.arange(0,time_steps*3,3)
    n=31
    yind1 = y_i
#     x_ind1 = 10
    # wnumber = 1
    range11 = np.max(np.abs((true_u_flux[:,-time_steps:,yind1,x_ind1])))/2.5
    levels = np.linspace(-range11, range11, n+1)
    cax_mean_online2 =axes[0,0].contourf(time,p[lowest_level:upper_level_p],true_field[lowest_level:upper_level_p,time_ind1:time_ind2,yind1,x_ind1],levels = levels,cmap=cmap1, extend="both")
    axes[0, 0].set_ylim(np.amax(p[lowest_level]), np.amin(p))
    if plot_trop:
        axes[0, 0].plot(time,trop_p)

    if change_range == 1:
        range11 = np.max(np.abs((pred_field[:,-time_steps:,yind1,x_ind1])))/2.5
        levels = np.linspace(-range11, range11, n+1)
        upper_level_p = pred_field.shape[0]
    cax_mean_online3 =axes[1,0].contourf(time,p[lowest_level:upper_level_p],pred_field[lowest_level:upper_level_p,time_ind1:time_ind2,yind1,x_ind1],levels = levels,cmap=cmap1, extend="both")
    axes[1, 0].set_ylim(np.amax(p[lowest_level]), np.amin(p))
    if plot_trop:
        axes[1, 0].plot(time,trop_p)

    


    color = 'black'
    lns1 =axes[2,0].plot(time,precip[yind1,x_ind1,time_ind1:time_ind2], color=color,label='precipitation')
    axes[2, 0].set_xlim(0, time[-1])
    axes[2, 0].tick_params(axis='y', labelcolor=color)


    ax2 = axes[2, 0].twinx()
    color = 'tab:blue'
    ax2.set_ylabel(r'Subgrid flux [kg m$^{-1}$ s$^{-2}$]', color=color,fontsize=fontsize)  # we already handled the x-label with ax1
    lns2 =ax2.plot(time,true_field_int_z[time_ind1:time_ind2,yind1,x_ind1], color=color,label='integrated flux (true)')
    ax2.tick_params(axis='y', labelcolor=color)
    lns3 =ax2.plot(time,pred_field_int_z[time_ind1:time_ind2,yind1,x_ind1], 'r:',label='integrated flux (predicted)')
    
    # added these three lines
    lns = lns1+lns2+lns3
    labs = [l.get_label() for l in lns]
    axes[2, 0].legend(lns, labs, loc=0,frameon=False)
    
    # yind1 = 43
    # # wnumber = 1
    # range11 = np.max(np.abs((fuz[:,yind1,x_ind1,:])))/1.4
    # levels = np.linspace(-range11, range11, n+1)
    # cax_mean_online3 =axes[0,1].contourf(time,p[lowest_level:upper_level_p],(fuz[lowest_level:upper_level_p,yind1,x_ind1,:]),levels = levels,cmap=cmap1)
    # axes[0, 1].set_ylim(np.amax(p[lowest_level]), np.amin(p))


    # color = 'tab:red'
    # cax_mean_online4 =axes[1,1].plot(time,precip11[yind1,x_ind1,:]*24*3600, color=color)
    # axes[1, 1].set_xlim(0, time[-1])
    # axes[1, 1].tick_params(axis='y', labelcolor=color)


    # ax2 = axes[1, 1].twinx()
    # color = 'tab:blue'
    # ax2.set_ylabel('subg flux', color=color)  # we already handled the x-label with ax1
    # cax_mean_online4 =ax2.plot(time,fuz_int[yind1,x_ind1,:], color=color)
    # ax2.tick_params(axis='y', labelcolor=color)



    x0 = axes[0,0].get_position().x0
    x1 = axes[0,0].get_position().x1
    y0 = axes[0,0].get_position().y0
    y1 = axes[0,0].get_position().y1

    for i in range(row_dim):
        for j  in range(col_dim):
            axes[i,j].set_position([x0 + j*fig_witdth + j*fig_h_dist,y0- i*fig_height - i*fig_v_dist,fig_witdth,fig_height])


    ind_let = 0
    for i in range(row_dim):
        for j  in range(col_dim):
            atext2 = axes[i, j].text(axes[0,0].get_position().x0*1.1+ j*fig_witdth + j*fig_h_dist,axes[0,0].get_position().y0*0.76- (i-1)*fig_height - (i-1)*fig_v_dist ,letters[ind_let],transform = fig.transFigure,color="black",fontsize=fontsize)

    #         header= axes[i, j].text(axes[0,0].get_position().x0*1.5+ j*fig_witdth + j*fig_h_dist,axes[0,0].get_position().y0*0.92- (i-1)*fig_height - (i-1)*fig_v_dist ,headers[ind_let],transform = fig.transFigure,color="black",fontsize=fontsize)
            ind_let = ind_let + 1

    pos1_rsq = axes[0, 0].get_position() # get the original position 
    cbaxes_rsq = fig.add_axes([pos1_rsq.x0+pos1_rsq.width + h_width_cbar, pos1_rsq.y0, colorbar_width, fig_height]) 
    cbar_rsq = fig.colorbar(cax_mean_online2,cax = cbaxes_rsq, ax=axes[0,0], shrink=1.0,ticks=flux_ticks_colorbar)
    cbar_rsq.ax.tick_params(labelsize=fontsize) 
    cbar_rsq.set_label(r'[kg m$^{-1}$ sec$^{-2}$]',fontsize=fontsize)
    
    if change_range == 1:
        flux_ticks_colorbar = np.arange(-range11,range11,range11/10)
    pos1_rsq = axes[1, 0].get_position() # get the original position 
    cbaxes_rsq = fig.add_axes([pos1_rsq.x0+pos1_rsq.width + h_width_cbar, pos1_rsq.y0, colorbar_width, fig_height]) 
#     cbar_rsq = fig.colorbar(cax_mean_online3,cax = cbaxes_rsq, ax=axes[1,0], shrink=1.0,ticks=flux_ticks_colorbar)
    cbar_rsq = fig.colorbar(cax_mean_online3,cax = cbaxes_rsq, ax=axes[1,0], shrink=1.0,ticks=flux_ticks_colorbar)
    cbar_rsq.ax.tick_params(labelsize=fontsize) 
    cbar_rsq.set_label(r'[kg m$^{-1}$ sec$^{-2}$]',fontsize=fontsize)



    # pos1_rsq = axes[0, 1].get_position() # get the original position 
    # cbaxes_rsq = fig.add_axes([pos1_rsq.x0+pos1_rsq.width + h_width_cbar, pos1_rsq.y0, colorbar_width, fig_height]) 
    # cbar_rsq = fig.colorbar(cax_mean_online3,cax = cbaxes_rsq, ax=axes[0,1], shrink=1.0,)

    # pos1_rsq = axes[1, 1].get_position() # get the original position 
    # cbaxes_rsq = fig.add_axes([pos1_rsq.x0+pos1_rsq.width + h_width_cbar, pos1_rsq.y0, colorbar_width, fig_height]) 
    # cbar_rsq = fig.colorbar(cax_mean_online4,cax = cbaxes_rsq, ax=axes[1,1], shrink=1.0,)





    # pos1_rsq = axes[1, 0].get_position() # get the original position 
    # cbaxes_rsq = fig.add_axes([pos1_rsq.x0+pos1_rsq.width + h_width_cbar, pos1_rsq.y0, colorbar_width, fig_height]) 
    # cbar_rsq = fig.colorbar(cax_mean_online5,cax = cbaxes_rsq, ax=axes[1,0], shrink=1.0,)





    axes[2, 0].set_xlabel('Time (hours)',fontsize=fontsize)
    axes[1, 1].set_xlabel('Time (hours)',fontsize=fontsize)
    axes[2, 0].set_ylabel(r'Precipitation  [mm day$^{-1}$]',fontsize=fontsize)
    axes[0, 0].set_ylabel('Pressure [hPa]',fontsize=fontsize)
    axes[1, 0].set_ylabel('Pressure [hPa]',fontsize=fontsize)


    # axes[0, 0].set_yticks(ticks_pmean)
    axes[0, 1].set_yticklabels([])
    # axes[1, 1].set_yticklabels([])


    
    axes[2,0].spines["top"].set_visible(False)
    axes[2,0].spines["right"].set_visible(False)

#     precip_ticks = [0,50,100,150,200,250,300,350,400,450]  
    axes[2, 0].set_ylim(0,400)
    ax2.set_ylim(-7000,8000)
    precip_ticks_label = [0,50,100,150]
    precip_ticks = precip_ticks_label
    axes[2, 0].set_yticks(precip_ticks)
    axes[2, 0].set_yticklabels(precip_ticks_label,fontsize=fontsize)

#     flux_ticks = [-2000,0,2000,4000,6000,8000]
#     flux_ticks = [-6000,-3000,0,3000,6000]
    flux_ticks_label = [0,3000,6000]
    flux_ticks = flux_ticks_label
    ax2.set_yticks(flux_ticks_label)
    ax2.set_yticklabels(flux_ticks_label,fontsize=fontsize)

    pressure_ticks = [300,600,900]
    time_ticks = [0,120,240]
    axes[0, 0].set_xticks(time_ticks)
    axes[0, 0].set_xticklabels([])
    axes[1, 0].set_xticks(time_ticks)
    axes[1, 0].set_xticklabels([])
    axes[2, 0].set_xticks(time_ticks)
    axes[2, 0].set_xticklabels(time_ticks,fontsize=fontsize)

    axes[0, 0].set_yticks(pressure_ticks)
    axes[0, 0].set_yticklabels(pressure_ticks,fontsize=fontsize)
    axes[1, 0].set_yticks(pressure_ticks)
    axes[1, 0].set_yticklabels(pressure_ticks,fontsize=fontsize)

    fig.delaxes(axes[0,1])
    fig.delaxes(axes[1,1])
    fig.delaxes(axes[2,1])

    save_path = '/glade/work/janniy/data_for_offline_momentum_paper/plots/'
    if save_flag:
        plt.savefig(save_path+save_name1 + '.eps', format='eps', bbox_inches="tight")
    
#     plt.savefig(save_path+'snapshot_u_flux_precip.eps', format='eps', bbox_inches="tight")
#     plt.savefig(save_path+'snapshot_u_flux_T_flux_show_noise2.eps', format='eps', bbox_inches="tight")
    # plt.savefig(save_path+'correcting_u_precip_subgrid_hovemoller_tropics2_int_flux.eps', format='eps', bbox_inches="tight")

In [91]:
def plot_sapshots_U_T_Q(true_field_U,true_field_T,true_field_Q,p,y_i =80,
                        change_range=0,save_flag = False,save_name1= 'del_snap',
                        plot_trop=False,trop_p = [],colorbar_flex=False,cosutm_ticks = False, 
                        tick1=[], tick2=[], tick3=[],time_ind1=-100,time_ind2=-1):
    per_day = 3600*24
    fontsize = 15
    fontsize_legend = 8
    h_width = 0.1
    v_width = 0.1
    h_width_cbar = 0.02
    colorbar_width =0.02

    row_dim = 3
    col_dim = 2

    fig_witdth = 1.
    fig_height = 0.75
    fig_h_dist = 0.2
    fig_v_dist = 0.2
    
    if cosutm_ticks:
        flux_ticks_colorbar = tick1
        flux_ticks_colorbarT = tick2
        flux_ticks_colorbarQ = tick3
    else:
        flux_ticks_colorbar = np.arange(-0.6,0.65,0.2)
        flux_ticks_colorbarT = np.arange(-0.6,0.65,0.2)
        flux_ticks_colorbarQ = np.arange(-0.0006,0.00065,0.0002)
    
    


    letters = ["(a) Subgrid flux of zonal momentum","(b) y_i = 43","(b) Subgrid flux of energy","(d)",'(c) Subgrid flux of non-precipitating water','','','','','']

    size1 = 2
    fig, axes = plt.subplots(row_dim,col_dim)
    # fig.set_figheight(10)
    # fig.set_figwidth(10)
    upper_level_p = 47
    lowest_level = 0
    cmap1 = 'bwr'
    time_steps = time_ind2-time_ind1
    time = np.arange(0,time_steps*3,3)
    n=31
    yind1 = y_i
    x_ind1 = 10
    # wnumber = 1
    if cosutm_ticks:
        range11 = np.max(tick1)
    else:
        range11 = np.max(np.abs((true_u_flux[:,-time_steps:,yind1,x_ind1])))/1.5

    levels = np.linspace(-range11, range11, n+1)
    cax_mean_online2 =axes[0,0].contourf(time,p[lowest_level:upper_level_p],true_field_U[lowest_level:upper_level_p,time_ind1:time_ind2,yind1,x_ind1],levels = levels,cmap=cmap1, extend="both")
    axes[0, 0].set_ylim(np.amax(p[lowest_level]), np.amin(p))
    if plot_trop:
        axes[0, 0].plot(time,trop_p)

    if cosutm_ticks:
        range11 = np.max(tick2)
    else:
        range11 = np.max(np.abs((true_field_T[:,-time_steps:,yind1,x_ind1])))/1.5
    levels = np.linspace(-range11, range11, n+1)
    cax_mean_online3 =axes[1,0].contourf(time,p[lowest_level:upper_level_p],true_field_T[lowest_level:upper_level_p,time_ind1:time_ind2,yind1,x_ind1],levels = levels,cmap=cmap1, extend="both")
    axes[1, 0].set_ylim(np.amax(p[lowest_level]), np.amin(p))
    if plot_trop:
        axes[1, 0].plot(time,trop_p)
    if cosutm_ticks:
        range11 = np.max(tick3)
    else:
        range11 = np.max(np.abs((true_field_Q[:,-time_steps:,yind1,x_ind1])))/1.5
    levels = np.linspace(-range11, range11, n+1)
    cax_mean_online4 =axes[2,0].contourf(time,p[lowest_level:upper_level_p],true_field_Q[lowest_level:upper_level_p,time_ind1:time_ind2,yind1,x_ind1],levels = levels,cmap=cmap1, extend="both")
    axes[2, 0].set_ylim(np.amax(p[lowest_level]), np.amin(p))

    
#     if change_range == 1:
#         range11 = np.max(np.abs((pred_field[:,-time_steps:,yind1,x_ind1])))/2.5
#         levels = np.linspace(-range11, range11, n+1)
#         upper_level_p = pred_field.shape[0]
#     cax_mean_online3 =axes[1,0].contourf(time,p[lowest_level:upper_level_p],pred_field[lowest_level:upper_level_p,-time_steps:,yind1,x_ind1],levels = levels,cmap=cmap1, extend="both")
#     axes[1, 0].set_ylim(np.amax(p[lowest_level]), np.amin(p))

    x0 = axes[0,0].get_position().x0
    x1 = axes[0,0].get_position().x1
    y0 = axes[0,0].get_position().y0
    y1 = axes[0,0].get_position().y1

    for i in range(row_dim):
        for j  in range(col_dim):
            axes[i,j].set_position([x0 + j*fig_witdth + j*fig_h_dist,y0- i*fig_height - i*fig_v_dist,fig_witdth,fig_height])

    ind_let = 0
    for i in range(row_dim):
        for j  in range(col_dim):
            atext2 = axes[i, j].text(axes[0,0].get_position().x0*1.1+ j*fig_witdth + j*fig_h_dist,axes[0,0].get_position().y0*0.76- (i-1)*fig_height - (i-1)*fig_v_dist ,letters[ind_let],transform = fig.transFigure,color="black",fontsize=fontsize)

    #         header= axes[i, j].text(axes[0,0].get_position().x0*1.5+ j*fig_witdth + j*fig_h_dist,axes[0,0].get_position().y0*0.92- (i-1)*fig_height - (i-1)*fig_v_dist ,headers[ind_let],transform = fig.transFigure,color="black",fontsize=fontsize)
            ind_let = ind_let + 1

    pos1_rsq = axes[0, 0].get_position() # get the original position 
    cbaxes_rsq = fig.add_axes([pos1_rsq.x0+pos1_rsq.width + h_width_cbar, pos1_rsq.y0, colorbar_width, fig_height]) 
    
    if colorbar_flex:
        range11 = np.max(np.abs((true_u_flux[:,-time_steps:,yind1,x_ind1])))/1.5
        levels = np.linspace(-range11, range11, n+1)
        cbar_rsq = fig.colorbar(cax_mean_online2,cax = cbaxes_rsq, ax=axes[1,0], shrink=1.0,ticks=levels[::10])
        cbar_rsq.set_ticklabels(levels[::10])
        print(levels[::10])
    else:
        cbar_rsq = fig.colorbar(cax_mean_online2,cax = cbaxes_rsq, ax=axes[0,0], shrink=1.0,ticks=flux_ticks_colorbar)
    cbar_rsq.ax.tick_params(labelsize=fontsize) 
    cbar_rsq.set_label(r'[kg m$^{-1}$ sec$^{-2}$]',fontsize=fontsize)
#     cbar_rsq.set_label(r'[kg m$^{-1}$ sec$^{-1}]$', rotation=270,fontsize=fontsize)

#     if change_range == 1:
#         flux_ticks_colorbar = np.arange(-range11,range11,range11/10)
    pos1_rsq = axes[1, 0].get_position() # get the original position 
    cbaxes_rsq = fig.add_axes([pos1_rsq.x0+pos1_rsq.width + h_width_cbar, pos1_rsq.y0, colorbar_width, fig_height]) 
#     cbar_rsq = fig.colorbar(cax_mean_online3,cax = cbaxes_rsq, ax=axes[1,0], shrink=1.0,ticks=flux_ticks_colorbar)
    
    if colorbar_flex:
        range11 = np.max(np.abs((true_field_T[:,-time_steps:,yind1,x_ind1])))/1.5
        levels = np.linspace(-range11, range11, n+1)
        cbar_rsq = fig.colorbar(cax_mean_online3,cax = cbaxes_rsq, ax=axes[1,0], shrink=1.0,ticks=levels[::10])
        cbar_rsq.set_ticklabels(levels[::10])
    else:
        cbar_rsq = fig.colorbar(cax_mean_online3,cax = cbaxes_rsq, ax=axes[1,0], shrink=1.0,ticks=flux_ticks_colorbarT)
    cbar_rsq.ax.tick_params(labelsize=fontsize)
    cbar_rsq.set_label(r'[kg m$^{-2}$ sec$^{-1}$ K]',fontsize=fontsize)
#     cbar_rsq.set_label(r'kg m$^{-1}$ sec$^{-1}$', rotation=270,fontsize=fontsize)

#     if change_range == 1:
#         flux_ticks_colorbar = np.arange(-range11,range11,range11/10)
    pos1_rsq = axes[2, 0].get_position() # get the original position 
    cbaxes_rsq = fig.add_axes([pos1_rsq.x0+pos1_rsq.width + h_width_cbar, pos1_rsq.y0, colorbar_width, fig_height]) 
    cbar_rsq = fig.colorbar(cax_mean_online4,cax = cbaxes_rsq, ax=axes[2,0], shrink=1.0,ticks=flux_ticks_colorbarQ)
    if colorbar_flex:
        range11 = np.max(np.abs((true_field_Q[:,-time_steps:,yind1,x_ind1])))/1.5
        levels = np.linspace(-range11, range11, n+1)
        cbar_rsq = fig.colorbar(cax_mean_online4,cax = cbaxes_rsq, ax=axes[1,0], shrink=1.0,ticks=levels[::10])
        cbar_rsq.set_ticklabels(levels[::10])
    else:
        cbar_rsq = fig.colorbar(cax_mean_online4,cax = cbaxes_rsq, ax=axes[2,0], shrink=1.0,ticks=flux_ticks_colorbarQ)

    cbar_rsq.ax.tick_params(labelsize=fontsize) 
    cbar_rsq.set_label(r'[kg m$^{-2}$ sec$^{-1}$ kg kg$^{-1}$]',fontsize=fontsize)




    axes[2, 0].set_xlabel('Time (hours)',fontsize=fontsize)
    axes[1, 1].set_xlabel('Time (hours)',fontsize=fontsize)
    axes[2, 0].set_ylabel(r'Pressure [hPa]',fontsize=fontsize)
    axes[0, 0].set_ylabel('Pressure [hPa]',fontsize=fontsize)
    axes[1, 0].set_ylabel('Pressure [hPa]',fontsize=fontsize)


    # axes[0, 0].set_yticks(ticks_pmean)
    # axes[1, 1].set_yticklabels([])


    
#     precip_ticks = [0,50,100,150,200,250,300,350,400,450]  
#     precip_ticks_label = [0,50,100,150]
#     precip_ticks = precip_ticks_label
#     axes[2, 0].set_yticks(precip_ticks)
#     axes[2, 0].set_yticklabels(precip_ticks_label,fontsize=fontsize)

# #     flux_ticks = [-2000,0,2000,4000,6000,8000]
# #     flux_ticks = [-6000,-3000,0,3000,6000]
#     flux_ticks_label = [0,3000,6000]
#     flux_ticks = flux_ticks_label
#     ax2.set_yticks(flux_ticks_label)
#     ax2.set_yticklabels(flux_ticks_label,fontsize=fontsize)

    pressure_ticks = [300,600,900]
    time_ticks = [0,120,240]
    axes[0, 0].set_xticks(time_ticks)
    axes[0, 0].set_xticklabels([])
    axes[1, 0].set_xticks(time_ticks)
    axes[1, 0].set_xticklabels([])
    axes[2, 0].set_xticks(time_ticks)
    axes[2, 0].set_xticklabels(time_ticks,fontsize=fontsize)

    axes[0, 0].set_yticks(pressure_ticks)
    axes[0, 0].set_yticklabels(pressure_ticks,fontsize=fontsize)
    axes[1, 0].set_yticks(pressure_ticks)
    axes[1, 0].set_yticklabels(pressure_ticks,fontsize=fontsize)
    axes[2, 0].set_yticks(pressure_ticks)
    axes[2, 0].set_yticklabels(pressure_ticks,fontsize=fontsize)

    fig.delaxes(axes[0,1])
    fig.delaxes(axes[1,1])
    fig.delaxes(axes[2,1])

    save_path = '/glade/work/janniy/data_for_offline_momentum_paper/plots/'
    if save_flag:
        plt.savefig(save_path+save_name1+'.eps', format='eps', bbox_inches="tight")    
#     plt.savefig(save_path+'snapshot_u_flux_precip.eps', format='eps', bbox_inches="tight")
#     plt.savefig(save_path+'snapshot_u_flux_T_flux_show_noise2.eps', format='eps', bbox_inches="tight")
    # plt.savefig(save_path+'correcting_u_precip_subgrid_hovemoller_tropics2_int_flux.eps', format='eps', bbox_inches="tight")

In [92]:
def plot_sapshots_U_T_Q_vs_x(true_field_U,true_field_T,true_field_Q,p,y_i =80,change_range=0,save_flag = False,save_name1= 'del_snap'):
    per_day = 3600*24
    fontsize = 15
    fontsize_legend = 8
    h_width = 0.1
    v_width = 0.1
    h_width_cbar = 0.02
    colorbar_width =0.02

    row_dim = 3
    col_dim = 2

    fig_witdth = 1.
    fig_height = 0.75
    fig_h_dist = 0.2
    fig_v_dist = 0.2
    
    flux_ticks_colorbar = np.arange(-0.6,0.65,0.2)
    flux_ticks_colorbarT = np.arange(-0.6,0.65,0.2)
    flux_ticks_colorbarQ = np.arange(-0.0006,0.00065,0.0002)

    letters = ["(a) Subgrid flux of zonal momentum","(b) y_i = 43","(b) Subgrid flux of energy","(d)",'(c) Subgrid flux of non-precipitating water','','','','','']

    size1 = 2
    fig, axes = plt.subplots(row_dim,col_dim)
    # fig.set_figheight(10)
    # fig.set_figwidth(10)
    upper_level_p = 47
    lowest_level = 0
    cmap1 = 'bwr'
    time_steps = 72
    x_x = np.arange(0,time_steps*96,96)
    n=20 
    yind1 = y_i
    t_ind1 = 120
    # wnumber = 1
    range11 = np.max(np.abs((true_u_flux[:,t_ind1,yind1,:])))/2.5
    levels = np.linspace(-range11, range11, n+1)
    cax_mean_online2 =axes[0,0].contourf(x_x,p[lowest_level:upper_level_p],true_field_U[lowest_level:upper_level_p,t_ind1,yind1,:],levels = levels,cmap=cmap1, extend="both")
    axes[0, 0].set_ylim(np.amax(p[lowest_level]), np.amin(p))

    
    range11 = np.max(np.abs((true_field_T[:,t_ind1,yind1,:])))/2.5
    levels = np.linspace(-range11, range11, n+1)
    cax_mean_online3 =axes[1,0].contourf(x_x,p[lowest_level:upper_level_p],true_field_T[lowest_level:upper_level_p,t_ind1,yind1,:],levels = levels,cmap=cmap1, extend="both")
    axes[1, 0].set_ylim(np.amax(p[lowest_level]), np.amin(p))

    range11 = np.max(np.abs((true_field_Q[:,t_ind1,yind1,:])))/2.5
    levels = np.linspace(-range11, range11, n+1)
    cax_mean_online4 =axes[2,0].contourf(x_x,p[lowest_level:upper_level_p],true_field_Q[lowest_level:upper_level_p,t_ind1,yind1,:],levels = levels,cmap=cmap1, extend="both")
    axes[2, 0].set_ylim(np.amax(p[lowest_level]), np.amin(p))

    
#     if change_range == 1:
#         range11 = np.max(np.abs((pred_field[:,-time_steps:,yind1,x_ind1])))/2.5
#         levels = np.linspace(-range11, range11, n+1)
#         upper_level_p = pred_field.shape[0]
#     cax_mean_online3 =axes[1,0].contourf(time,p[lowest_level:upper_level_p],pred_field[lowest_level:upper_level_p,-time_steps:,yind1,x_ind1],levels = levels,cmap=cmap1, extend="both")
#     axes[1, 0].set_ylim(np.amax(p[lowest_level]), np.amin(p))

    x0 = axes[0,0].get_position().x0
    x1 = axes[0,0].get_position().x1
    y0 = axes[0,0].get_position().y0
    y1 = axes[0,0].get_position().y1

    for i in range(row_dim):
        for j  in range(col_dim):
            axes[i,j].set_position([x0 + j*fig_witdth + j*fig_h_dist,y0- i*fig_height - i*fig_v_dist,fig_witdth,fig_height])

    ind_let = 0
    for i in range(row_dim):
        for j  in range(col_dim):
            atext2 = axes[i, j].text(axes[0,0].get_position().x0*1.1+ j*fig_witdth + j*fig_h_dist,axes[0,0].get_position().y0*0.76- (i-1)*fig_height - (i-1)*fig_v_dist ,letters[ind_let],transform = fig.transFigure,color="black",fontsize=fontsize)

    #         header= axes[i, j].text(axes[0,0].get_position().x0*1.5+ j*fig_witdth + j*fig_h_dist,axes[0,0].get_position().y0*0.92- (i-1)*fig_height - (i-1)*fig_v_dist ,headers[ind_let],transform = fig.transFigure,color="black",fontsize=fontsize)
            ind_let = ind_let + 1

    pos1_rsq = axes[0, 0].get_position() # get the original position 
    cbaxes_rsq = fig.add_axes([pos1_rsq.x0+pos1_rsq.width + h_width_cbar, pos1_rsq.y0, colorbar_width, fig_height]) 
    cbar_rsq = fig.colorbar(cax_mean_online2,cax = cbaxes_rsq, ax=axes[0,0], shrink=1.0,ticks=flux_ticks_colorbar)
    cbar_rsq.ax.tick_params(labelsize=fontsize) 
    cbar_rsq.set_label(r'[kg m$^{-1}$ sec$^{-2}$]',fontsize=fontsize)
#     cbar_rsq.set_label(r'[kg m$^{-1}$ sec$^{-1}]$', rotation=270,fontsize=fontsize)

#     if change_range == 1:
#         flux_ticks_colorbar = np.arange(-range11,range11,range11/10)
    pos1_rsq = axes[1, 0].get_position() # get the original position 
    cbaxes_rsq = fig.add_axes([pos1_rsq.x0+pos1_rsq.width + h_width_cbar, pos1_rsq.y0, colorbar_width, fig_height]) 
#     cbar_rsq = fig.colorbar(cax_mean_online3,cax = cbaxes_rsq, ax=axes[1,0], shrink=1.0,ticks=flux_ticks_colorbar)
    cbar_rsq = fig.colorbar(cax_mean_online3,cax = cbaxes_rsq, ax=axes[1,0], shrink=1.0,ticks=flux_ticks_colorbarT)
    cbar_rsq.ax.tick_params(labelsize=fontsize)
    cbar_rsq.set_label(r'[kg m$^{-2}$ sec$^{-1}$ kg kg$^{-1}$]',fontsize=fontsize)
#     cbar_rsq.set_label(r'kg m$^{-1}$ sec$^{-1}$', rotation=270,fontsize=fontsize)

#     if change_range == 1:
#         flux_ticks_colorbar = np.arange(-range11,range11,range11/10)
    pos1_rsq = axes[2, 0].get_position() # get the original position 
    cbaxes_rsq = fig.add_axes([pos1_rsq.x0+pos1_rsq.width + h_width_cbar, pos1_rsq.y0, colorbar_width, fig_height]) 
    cbar_rsq = fig.colorbar(cax_mean_online4,cax = cbaxes_rsq, ax=axes[2,0], shrink=1.0,ticks=flux_ticks_colorbarQ)
    cbar_rsq.ax.tick_params(labelsize=fontsize) 
    cbar_rsq.set_label(r'[kg m$^{-2}$ sec$^{-1}$ K]',fontsize=fontsize)



    axes[2, 0].set_xlabel('x (km)',fontsize=fontsize)
    axes[1, 1].set_xlabel('x (km)',fontsize=fontsize)
    axes[2, 0].set_ylabel(r'Pressure [hPa]',fontsize=fontsize)
    axes[0, 0].set_ylabel('Pressure [hPa]',fontsize=fontsize)
    axes[1, 0].set_ylabel('Pressure [hPa]',fontsize=fontsize)


    # axes[0, 0].set_yticks(ticks_pmean)
    # axes[1, 1].set_yticklabels([])


    
#     precip_ticks = [0,50,100,150,200,250,300,350,400,450]  
#     precip_ticks_label = [0,50,100,150]
#     precip_ticks = precip_ticks_label
#     axes[2, 0].set_yticks(precip_ticks)
#     axes[2, 0].set_yticklabels(precip_ticks_label,fontsize=fontsize)

# #     flux_ticks = [-2000,0,2000,4000,6000,8000]
# #     flux_ticks = [-6000,-3000,0,3000,6000]
#     flux_ticks_label = [0,3000,6000]
#     flux_ticks = flux_ticks_label
#     ax2.set_yticks(flux_ticks_label)
#     ax2.set_yticklabels(flux_ticks_label,fontsize=fontsize)

    pressure_ticks = [300,600,900]
    x_ticks = [0,96*18,96*36,96*54]
    axes[0, 0].set_xticks(x_ticks)
    axes[0, 0].set_xticklabels([])
    axes[1, 0].set_xticks(x_ticks)
    axes[1, 0].set_xticklabels([])
    axes[2, 0].set_xticks(x_ticks)
    axes[2, 0].set_xticklabels(x_ticks,fontsize=fontsize)

    axes[0, 0].set_yticks(pressure_ticks)
    axes[0, 0].set_yticklabels(pressure_ticks,fontsize=fontsize)
    axes[1, 0].set_yticks(pressure_ticks)
    axes[1, 0].set_yticklabels(pressure_ticks,fontsize=fontsize)
    axes[2, 0].set_yticks(pressure_ticks)
    axes[2, 0].set_yticklabels(pressure_ticks,fontsize=fontsize)

    fig.delaxes(axes[0,1])
    fig.delaxes(axes[1,1])
    fig.delaxes(axes[2,1])

    save_path = '/glade/work/janniy/data_for_offline_momentum_paper/plots/'
    if save_flag:
        plt.savefig(save_path+save_name1+'.eps', format='eps', bbox_inches="tight")    
#     plt.savefig(save_path+'snapshot_u_flux_precip.eps', format='eps', bbox_inches="tight")
#     plt.savefig(save_path+'snapshot_u_flux_T_flux_show_noise2.eps', format='eps', bbox_inches="tight")
    # plt.savefig(save_path+'correcting_u_precip_subgrid_hovemoller_tropics2_int_flux.eps', format='eps', bbox_inches="tight")

In [90]:
def coarse_var_x(variable,coarse_fact = 4):
    # I assume we coarse over dims 2,3
    X_size = int(variable.shape[3]/coarse_fact)
    var_coarse = np.zeros((variable.shape[0],variable.shape[1],variable.shape[2],X_size))
    for i in range(X_size):
        var_coarse[:,:,:,i] = np.mean(variable[:,:,:,i*coarse_fact:(i+1)*coarse_fact],axis = (3))
            
    return var_coarse

In [109]:
# I want to calculate the tropopause level (fr a specific time and location). 
# I might want to include the condition of several levels below the threshold. 
def calc_lapse_rate_time(T,z,p, min_lev_trop_ind=10):
#     # Calculating the index of tropopause height
    d_z1 = np.zeros(z.shape[0]-2)
    d_z1[:] = z[2:] - z[:-2]
    T_zonal_t_mean = T
    T_zonal_t_mean_diff = T_zonal_t_mean[2:,:]-T_zonal_t_mean[:-2,:]
    dT_dz = T_zonal_t_mean_diff[:,:]/d_z1[:,None]

# #     for yi in range(field.shape[1]):
# #         np.where(field[:,yi] > -0.002)
    trop_ind = np.zeros(T.shape[1])
    trop_z = np.zeros(T.shape[1])
    trop_p = np.zeros(T.shape[1])
    for yi in range(dT_dz.shape[1]):
        
        flag1 = dT_dz[min_lev_trop_ind:,yi] > -0.002
        flag2 = np.zeros(flag1.shape[0])
        for ii in range(flag1.shape[0]):
            if flag1[ii] and flag1[ii+1]and flag1[ii+2]:
                trop_ind[yi] = ii  + min_lev_trop_ind 
                trop_z[yi] = z[ii+min_lev_trop_ind ]
                trop_p[yi] = p[ii+min_lev_trop_ind ]
                break
                

                            
#         aa = np.where(dT_dz[min_lev_trop_ind:,yi] > -0.002)
        
#         trop_ind[yi] = min(aa)[0]  + min_lev_trop_ind 
#         trop_z[yi] = z[min(aa)[0]+min_lev_trop_ind ]
#         trop_p[yi] = p[min(aa)[0]+min_lev_trop_ind ]
        
    return trop_ind,trop_z,trop_p

In [77]:
def plot_sapshots_2x2_mom_energy(true_field,true_sec_field,pred_field,true_field_int_z,pred_field_int_z,
                  precip,p,y_i =80,change_range=0,save_flag = False,save_name1= 'del_snap',
                  plot_trop=False,trop_p = [],
                  x_ind1 =10, time_ind1=-100, time_ind2 = -1,
                                 flux_ticks_colorbarT = np.arange(-0.6,0.65,0.3),
                                 flux_ticks_colorbar = np.arange(-0.6,0.65,0.3),
                                 flag_moisture = False,
                                 flux_ticks_label = [-0.3,-0.15,0,0.15000,0.3000],
                                 y_lim1 =-0.7000, y_lim2 =0.4400,
                                 range_momentum = 0,
                                 range_energy = 0
                 ):
    per_day = 3600*24
    fontsize = 15
    fontsize_legend = 8
    h_width = 0.1
    v_width = 0.1
    h_width_cbar = 0.02
    colorbar_width =0.02

    row_dim = 3
    col_dim = 2

    fig_witdth = 0.75
    fig_height = 0.65
    fig_h_dist = 0.4
    fig_v_dist = 0.2
    
#     flux_ticks_colorbar = np.arange(-0.6,0.65,0.2)
#     flux_ticks_colorbar = np.arange(-0.6,0.65,0.2)

    if flag_moisture:
        letters = ["(a) True subgrid momentum flux","(b) Predicted subgrid momentum flux",'(c) Integrated fluxes and precipitation','(d) True subgrid moisture flux','','','','']
    else:
        letters = ["(a) True subgrid momentum flux","(b) Predicted subgrid momentum flux",'(c) Integrated fluxes and precipitation','(d) True subgrid energy flux','','','','']
#     letters = ["(a) True subgrid momentum flux","(b) Predicted subgrid momentum flux",'(c) Integrated fluxes and precipitation','(d) True subgrid moisture flux','','','','']

    size1 = 2
    fig, axes = plt.subplots(row_dim,col_dim)
    # fig.set_figheight(10)
    # fig.set_figwidth(10)
    upper_level_p = 47
    lowest_level = 0
    cmap1 = 'bwr'
    time_steps = time_ind2 - time_ind1
    time = np.arange(0,time_steps*3,3)
    n=31
    yind1 = y_i
#     x_ind1 = 10
    # wnumber = 1
    if range_momentum == 0:
        range11 = np.max(np.abs((true_u_flux[:,-time_steps:,yind1,x_ind1])))/1.5
    else:
        range11 = range_momentum
    levels = np.linspace(-range11, range11, n+1)
    print(time.shape)
    print(p[lowest_level:upper_level_p].shape)
    print(true_field[lowest_level:upper_level_p,time_ind1:time_ind2,yind1,x_ind1][lowest_level:upper_level_p].shape)
    cax_mean_online2 =axes[0,0].contourf(time,p[lowest_level:upper_level_p],true_field[lowest_level:upper_level_p,time_ind1:time_ind2,yind1,x_ind1],levels = levels,cmap=cmap1, extend="both")
    axes[0, 0].set_ylim(np.amax(p[lowest_level]), np.amin(p))
    if plot_trop:
        axes[0, 0].plot(time,trop_p)

    if change_range == 1:
        range11 = np.max(np.abs((pred_field[:,-time_steps:,yind1,x_ind1])))/1.5
        levels = np.linspace(-range11, range11, n+1)
        upper_level_p = pred_field.shape[0]
    cax_mean_online3 =axes[0,1].contourf(time,p[lowest_level:upper_level_p],pred_field[lowest_level:upper_level_p,time_ind1:time_ind2,yind1,x_ind1],levels = levels,cmap=cmap1, extend="both")
    axes[0, 1].set_ylim(np.amax(p[lowest_level]), np.amin(p))
    if plot_trop:
        axes[1, 0].plot(time,trop_p)

    


    color = 'black'
    lns1 =axes[1,0].plot(time,precip[yind1,x_ind1,time_ind1:time_ind2], color=color,label='precipitation')
    axes[1, 0].set_xlim(0, time[-1])
    axes[1, 0].tick_params(axis='y', labelcolor=color)


    ax2 = axes[1, 0].twinx()
    color = 'tab:blue'
    ax2.set_ylabel(r'Subgrid flux [kg m$^{-1}$ s$^{-2}$]', color=color,fontsize=fontsize)  # we already handled the x-label with ax1
#     ax2.set_ylabel(r'Subgrid flux [kg s$^{-2}$]', color=color,fontsize=fontsize)  # we already handled the x-label with ax1
    lns2 =ax2.plot(time,true_field_int_z[time_ind1:time_ind2,yind1,x_ind1], color=color,label='integrated flux (true)')
    ax2.tick_params(axis='y', labelcolor=color)
    lns3 =ax2.plot(time,pred_field_int_z[time_ind1:time_ind2,yind1,x_ind1], 'r:',label='integrated flux (predicted)')
    
    # added these three lines
    lns = lns1+lns2+lns3
    labs = [l.get_label() for l in lns]
    axes[1, 0].legend(lns, labs, loc=0,frameon=False)
    
    
    upper_level_p_energy = 30
    if range_energy ==0:
        range_energy = np.max(np.abs((true_sec_field[:,-time_steps:,yind1,x_ind1])))/1.5
    else:
        range_energy =range_energy
    levels_energy = np.linspace(-range_energy, range_energy, n+1)
    cax_mean_online_enrgy =axes[1,1].contourf(time,p[lowest_level:upper_level_p_energy],true_sec_field[lowest_level:upper_level_p_energy,time_ind1:time_ind2,yind1,x_ind1],levels = levels_energy,cmap=cmap1, extend="both")
    axes[1, 1].set_ylim(np.amax(p[lowest_level]), np.amin(p))
    if plot_trop:
        axes[1, 1].plot(time,trop_p)
    
    # yind1 = 43
    # # wnumber = 1
    # range11 = np.max(np.abs((fuz[:,yind1,x_ind1,:])))/1.4
    # levels = np.linspace(-range11, range11, n+1)
    # cax_mean_online3 =axes[0,1].contourf(time,p[lowest_level:upper_level_p],(fuz[lowest_level:upper_level_p,yind1,x_ind1,:]),levels = levels,cmap=cmap1)
    # axes[0, 1].set_ylim(np.amax(p[lowest_level]), np.amin(p))


    # color = 'tab:red'
    # cax_mean_online4 =axes[1,1].plot(time,precip11[yind1,x_ind1,:]*24*3600, color=color)
    # axes[1, 1].set_xlim(0, time[-1])
    # axes[1, 1].tick_params(axis='y', labelcolor=color)


    # ax2 = axes[1, 1].twinx()
    # color = 'tab:blue'
    # ax2.set_ylabel('subg flux', color=color)  # we already handled the x-label with ax1
    # cax_mean_online4 =ax2.plot(time,fuz_int[yind1,x_ind1,:], color=color)
    # ax2.tick_params(axis='y', labelcolor=color)



    x0 = axes[0,0].get_position().x0
    x1 = axes[0,0].get_position().x1
    y0 = axes[0,0].get_position().y0
    y1 = axes[0,0].get_position().y1

    for i in range(row_dim):
        for j  in range(col_dim):
            axes[i,j].set_position([x0 + j*fig_witdth + j*fig_h_dist,y0- i*fig_height - i*fig_v_dist,fig_witdth,fig_height])


    ind_let = 0
    for i in range(row_dim):
        for j  in range(col_dim):
            atext2 = axes[i, j].text(axes[0,0].get_position().x0*1.1+ j*fig_witdth + j*fig_h_dist,axes[0,0].get_position().y0*0.76- (i-1)*fig_height - (i-1)*fig_v_dist ,letters[ind_let],transform = fig.transFigure,color="black",fontsize=fontsize)

    #         header= axes[i, j].text(axes[0,0].get_position().x0*1.5+ j*fig_witdth + j*fig_h_dist,axes[0,0].get_position().y0*0.92- (i-1)*fig_height - (i-1)*fig_v_dist ,headers[ind_let],transform = fig.transFigure,color="black",fontsize=fontsize)
            ind_let = ind_let + 1

    pos1_rsq = axes[0, 0].get_position() # get the original position 
    cbaxes_rsq = fig.add_axes([pos1_rsq.x0+pos1_rsq.width + h_width_cbar, pos1_rsq.y0, colorbar_width, fig_height]) 
    cbar_rsq = fig.colorbar(cax_mean_online2,cax = cbaxes_rsq, ax=axes[0,0], shrink=1.0,ticks=flux_ticks_colorbar)
    cbar_rsq.ax.tick_params(labelsize=fontsize) 
    cbar_rsq.set_label(r'[kg m$^{-1}$ sec$^{-2}$]',fontsize=fontsize)
    
    if change_range == 1:
        flux_ticks_colorbar = np.arange(-range11,range11,range11/10)
    pos1_rsq = axes[0, 1].get_position() # get the original position 
    cbaxes_rsq = fig.add_axes([pos1_rsq.x0+pos1_rsq.width + h_width_cbar, pos1_rsq.y0, colorbar_width, fig_height]) 
#     cbar_rsq = fig.colorbar(cax_mean_online3,cax = cbaxes_rsq, ax=axes[1,0], shrink=1.0,ticks=flux_ticks_colorbar)
    cbar_rsq = fig.colorbar(cax_mean_online3,cax = cbaxes_rsq, ax=axes[1,0], shrink=1.0,ticks=flux_ticks_colorbar)
    cbar_rsq.ax.tick_params(labelsize=fontsize) 
    cbar_rsq.set_label(r'[kg m$^{-1}$ sec$^{-2}$]',fontsize=fontsize)

    pos1_rsq = axes[1, 1].get_position() # get the original position 
    cbaxes_rsq = fig.add_axes([pos1_rsq.x0+pos1_rsq.width + h_width_cbar, pos1_rsq.y0, colorbar_width, fig_height]) 
    cbar_rsq = fig.colorbar(cax_mean_online_enrgy,cax = cbaxes_rsq, ax=axes[1,1], shrink=1.0,ticks=flux_ticks_colorbarT)
    cbar_rsq.ax.tick_params(labelsize=fontsize) 
#     cbar_rsq.set_label(r'[kg m$^{-2}$ sec$^{-1}$ K]',fontsize=fontsize)
    if flag_moisture:
        cbar_rsq.set_label(r'[kg m$^{-2}$ sec$^{-1}$ kg$^{-1}$ kg]',fontsize=fontsize)
    else:
        cbar_rsq.set_label(r'[kg m$^{-2}$ sec$^{-1}$ K]',fontsize=fontsize)

    # pos1_rsq = axes[0, 1].get_position() # get the original position 
    # cbaxes_rsq = fig.add_axes([pos1_rsq.x0+pos1_rsq.width + h_width_cbar, pos1_rsq.y0, colorbar_width, fig_height]) 
    # cbar_rsq = fig.colorbar(cax_mean_online3,cax = cbaxes_rsq, ax=axes[0,1], shrink=1.0,)

    # pos1_rsq = axes[1, 1].get_position() # get the original position 
    # cbaxes_rsq = fig.add_axes([pos1_rsq.x0+pos1_rsq.width + h_width_cbar, pos1_rsq.y0, colorbar_width, fig_height]) 
    # cbar_rsq = fig.colorbar(cax_mean_online4,cax = cbaxes_rsq, ax=axes[1,1], shrink=1.0,)





    # pos1_rsq = axes[1, 0].get_position() # get the original position 
    # cbaxes_rsq = fig.add_axes([pos1_rsq.x0+pos1_rsq.width + h_width_cbar, pos1_rsq.y0, colorbar_width, fig_height]) 
    # cbar_rsq = fig.colorbar(cax_mean_online5,cax = cbaxes_rsq, ax=axes[1,0], shrink=1.0,)





    axes[1, 0].set_xlabel('Time (hours)',fontsize=fontsize)
    axes[1, 1].set_xlabel('Time (hours)',fontsize=fontsize)
    axes[1, 0].set_ylabel(r'Precipitation  [mm day$^{-1}$]',fontsize=fontsize)
    axes[0, 0].set_ylabel('Pressure [hPa]',fontsize=fontsize)
    axes[0, 1].set_ylabel('Pressure [hPa]',fontsize=fontsize)
    axes[1, 1].set_ylabel('Pressure [hPa]',fontsize=fontsize)


    # axes[0, 0].set_yticks(ticks_pmean)
    axes[0, 1].set_yticklabels([])
    # axes[1, 1].set_yticklabels([])


    
    axes[1,0].spines["top"].set_visible(False)
    axes[1,0].spines["right"].set_visible(False)

#     precip_ticks = [0,50,100,150,200,250,300,350,400,450]  
    axes[1, 0].set_ylim(0,400)
    
#     ax2.set_ylim(-7000,8000)
    ax2.set_ylim(y_lim1,y_lim2)
#     ax2.set_ylim(-0.3000,0.3400)
    precip_ticks_label = [0,50,100,150]
    precip_ticks = precip_ticks_label
    axes[1, 0].set_yticks(precip_ticks)
    axes[1, 0].set_yticklabels(precip_ticks_label,fontsize=fontsize)

#     flux_ticks = [-2000,0,2000,4000,6000,8000]
#     flux_ticks = [-6000,-3000,0,3000,6000]
#     flux_ticks_label = [0,0.15000,0.3000]
#     flux_ticks_label = [-0.3,-0.15,0,0.15000,0.3000]
    flux_ticks = flux_ticks_label
    ax2.set_yticks(flux_ticks_label)
    ax2.set_yticklabels(flux_ticks_label,fontsize=fontsize)

    pressure_ticks = [300,600,900]
    time_ticks = [0,120,240]
    axes[0, 0].set_xticks(time_ticks)
    axes[0, 0].set_xticklabels([])
    axes[1, 0].set_xticks(time_ticks)
    axes[1, 0].set_xticklabels([])
    axes[1, 1].set_xticks(time_ticks)
    axes[1, 1].set_xticklabels(time_ticks,fontsize=fontsize)
    axes[0, 1].set_xticks(time_ticks)
    axes[0, 1].set_xticklabels(time_ticks,fontsize=fontsize)

    
    axes[0, 0].set_yticks(pressure_ticks)
    axes[0, 0].set_yticklabels(pressure_ticks,fontsize=fontsize)
    axes[1, 1].set_yticks(pressure_ticks)
    axes[1, 1].set_yticklabels(pressure_ticks,fontsize=fontsize)
    axes[0, 1].set_yticks(pressure_ticks)
    axes[0, 1].set_yticklabels(pressure_ticks,fontsize=fontsize)

    fig.delaxes(axes[2,0])
    fig.delaxes(axes[2,1])

    save_path = '/glade/work/janniy/data_for_offline_momentum_paper/plots/'
    if save_flag:
        plt.savefig(save_path+save_name1 + '.eps', format='eps', bbox_inches="tight")
    
#     plt.savefig(save_path+'snapshot_u_flux_precip.eps', format='eps', bbox_inches="tight")
#     plt.savefig(save_path+'snapshot_u_flux_T_flux_show_noise2.eps', format='eps', bbox_inches="tight")
    # plt.savefig(save_path+'correcting_u_precip_subgrid_hovemoller_tropics2_int_flux.eps', format='eps', bbox_inches="tight")

In [78]:
def plot_sapshots_2x2_mom_energy_switch(true_field,true_sec_field,pred_field,true_field_int_z,pred_field_int_z,
                  precip,p,y_i =80,change_range=0,save_flag = False,save_name1= 'del_snap',
                  plot_trop=False,trop_p = [],
                  x_ind1 =10, time_ind1=-100, time_ind2 = -1,
                                 flux_ticks_colorbarT = np.arange(-0.6,0.65,0.3),
                                 flux_ticks_colorbar = np.arange(-0.6,0.65,0.3),
                                 flag_moisture = False,
                                 flux_ticks_label = [-0.3,-0.15,0,0.15000,0.3000],
                                 y_lim1 =-0.7000, y_lim2 =0.4400,
                                 range_momentum = 0,
                                 range_energy = 0, sat_fact = 1.5, precip_labels = [0,50,100,150],
                                 letters = ["(a) True subgrid momentum flux","(b) Mean fluxes and precipitation",'(c) Predicted subgrid momentum flux','(d) True subgrid energy flux','','','','']
                 ):
    per_day = 3600*24
    fontsize = 15
    fontsize_legend = 8
    h_width = 0.1
    v_width = 0.1
    h_width_cbar = 0.02
    colorbar_width =0.02

    row_dim = 3
    col_dim = 2

    fig_witdth = 0.75
    fig_height = 0.65
    fig_h_dist = 0.4
    fig_v_dist = 0.2
    
#     flux_ticks_colorbar = np.arange(-0.6,0.65,0.2)
#     flux_ticks_colorbar = np.arange(-0.6,0.65,0.2)

#     if flag_moisture:
#         letters = ["(a) True subgrid momentum flux","(b) Mean fluxes and precipitation",'(c) Predicted subgrid momentum flux','(d) True subgrid moisture flux','','','','']
#     else:
#         letters = ["(a) True subgrid momentum flux","(b) Mean fluxes and precipitation",'(c) Predicted subgrid momentum flux','(d) True subgrid energy flux','','','','']
#     letters = ["(a) True subgrid momentum flux","(b) Predicted subgrid momentum flux",'(c) Integrated fluxes and precipitation','(d) True subgrid moisture flux','','','','']

    size1 = 2
    fig, axes = plt.subplots(row_dim,col_dim)
    # fig.set_figheight(10)
    # fig.set_figwidth(10)
    upper_level_p = 47
    lowest_level = 0
    cmap1 = 'bwr'
    time_steps = time_ind2 - time_ind1
    time = np.arange(0,time_steps*3,3)
    n=31
    yind1 = y_i
#     x_ind1 = 10
    # wnumber = 1
    if range_momentum == 0:
        range11 = np.max(np.abs((true_u_flux[:,-time_steps:,yind1,x_ind1])))/sat_fact
    else:
        range11 = range_momentum
    levels = np.linspace(-range11, range11, n+1)
    print(time.shape)
    print(p[lowest_level:upper_level_p].shape)
    print(true_field[lowest_level:upper_level_p,time_ind1:time_ind2,yind1,x_ind1][lowest_level:upper_level_p].shape)
    cax_mean_online2 =axes[0,0].contourf(time,p[lowest_level:upper_level_p],true_field[lowest_level:upper_level_p,time_ind1:time_ind2,yind1,x_ind1],levels = levels,cmap=cmap1, extend="both")
    axes[0, 0].set_ylim(np.amax(p[lowest_level]), np.amin(p))
    if plot_trop:
        axes[0, 0].plot(time,trop_p)

    if change_range == 1:
        range11 = np.max(np.abs((pred_field[:,-time_steps:,yind1,x_ind1])))/sat_fact
        levels = np.linspace(-range11, range11, n+1)
        upper_level_p = pred_field.shape[0]
    cax_mean_online3 =axes[1,0].contourf(time,p[lowest_level:upper_level_p],pred_field[lowest_level:upper_level_p,time_ind1:time_ind2,yind1,x_ind1],levels = levels,cmap=cmap1, extend="both")
    axes[1, 0].set_ylim(np.amax(p[lowest_level]), np.amin(p))
    if plot_trop:
        axes[1, 0].plot(time,trop_p)

    


    color = 'black'
    lns1 =axes[0,1].plot(time,precip[yind1,x_ind1,time_ind1:time_ind2], color=color,label='precipitation')
    axes[0, 1].set_xlim(0, time[-1])
    axes[0, 1].tick_params(axis='y', labelcolor=color)


    ax2 = axes[0, 1].twinx()
    color = 'tab:blue'
    ax2.set_ylabel(r'Subgrid flux [kg m$^{-1}$ s$^{-2}$]', color=color,fontsize=fontsize)  # we already handled the x-label with ax1
#     ax2.set_ylabel(r'Subgrid flux [kg s$^{-2}$]', color=color,fontsize=fontsize)  # we already handled the x-label with ax1
    lns2 =ax2.plot(time,true_field_int_z[time_ind1:time_ind2,yind1,x_ind1], color=color,label='mean flux (true)')
    ax2.tick_params(axis='y', labelcolor=color)
    lns3 =ax2.plot(time,pred_field_int_z[time_ind1:time_ind2,yind1,x_ind1], 'r:',label='mean flux (predicted)')
    
    # added these three lines
    lns = lns1+lns2+lns3
    labs = [l.get_label() for l in lns]
    axes[0, 1].legend(lns, labs, loc='upper center',frameon=False)
    
    
    upper_level_p_energy = 30
    if range_energy ==0:
        range_energy = np.max(np.abs((true_sec_field[:,-time_steps:,yind1,x_ind1])))/1.5
    else:
        range_energy =range_energy
    levels_energy = np.linspace(-range_energy, range_energy, n+1)
    cax_mean_online_enrgy =axes[1,1].contourf(time,p[lowest_level:upper_level_p_energy],true_sec_field[lowest_level:upper_level_p_energy,time_ind1:time_ind2,yind1,x_ind1],levels = levels_energy,cmap=cmap1, extend="both")
    axes[1, 1].set_ylim(np.amax(p[lowest_level]), np.amin(p))
    if plot_trop:
        axes[1, 1].plot(time,trop_p)
    
    # yind1 = 43
    # # wnumber = 1
    # range11 = np.max(np.abs((fuz[:,yind1,x_ind1,:])))/1.4
    # levels = np.linspace(-range11, range11, n+1)
    # cax_mean_online3 =axes[0,1].contourf(time,p[lowest_level:upper_level_p],(fuz[lowest_level:upper_level_p,yind1,x_ind1,:]),levels = levels,cmap=cmap1)
    # axes[0, 1].set_ylim(np.amax(p[lowest_level]), np.amin(p))


    # color = 'tab:red'
    # cax_mean_online4 =axes[1,1].plot(time,precip11[yind1,x_ind1,:]*24*3600, color=color)
    # axes[1, 1].set_xlim(0, time[-1])
    # axes[1, 1].tick_params(axis='y', labelcolor=color)


    # ax2 = axes[1, 1].twinx()
    # color = 'tab:blue'
    # ax2.set_ylabel('subg flux', color=color)  # we already handled the x-label with ax1
    # cax_mean_online4 =ax2.plot(time,fuz_int[yind1,x_ind1,:], color=color)
    # ax2.tick_params(axis='y', labelcolor=color)



    x0 = axes[0,0].get_position().x0
    x1 = axes[0,0].get_position().x1
    y0 = axes[0,0].get_position().y0
    y1 = axes[0,0].get_position().y1

    for i in range(row_dim):
        for j  in range(col_dim):
            axes[i,j].set_position([x0 + j*fig_witdth + j*fig_h_dist,y0- i*fig_height - i*fig_v_dist,fig_witdth,fig_height])


    ind_let = 0
    for i in range(row_dim):
        for j  in range(col_dim):
            atext2 = axes[i, j].text(axes[0,0].get_position().x0*1.1+ j*fig_witdth + j*fig_h_dist,axes[0,0].get_position().y0*0.76- (i-1)*fig_height - (i-1)*fig_v_dist ,letters[ind_let],transform = fig.transFigure,color="black",fontsize=fontsize)

    #         header= axes[i, j].text(axes[0,0].get_position().x0*1.5+ j*fig_witdth + j*fig_h_dist,axes[0,0].get_position().y0*0.92- (i-1)*fig_height - (i-1)*fig_v_dist ,headers[ind_let],transform = fig.transFigure,color="black",fontsize=fontsize)
            ind_let = ind_let + 1

    pos1_rsq = axes[0, 0].get_position() # get the original position 
    cbaxes_rsq = fig.add_axes([pos1_rsq.x0+pos1_rsq.width + h_width_cbar, pos1_rsq.y0, colorbar_width, fig_height]) 
    cbar_rsq = fig.colorbar(cax_mean_online2,cax = cbaxes_rsq, ax=axes[0,0], shrink=1.0,ticks=flux_ticks_colorbar)
    cbar_rsq.ax.tick_params(labelsize=fontsize) 
    cbar_rsq.set_label(r'[kg m$^{-1}$ sec$^{-2}$]',fontsize=fontsize)
    
    if change_range == 1:
        flux_ticks_colorbar = np.arange(-range11,range11,range11/10)
    pos1_rsq = axes[1, 0].get_position() # get the original position 
    cbaxes_rsq = fig.add_axes([pos1_rsq.x0+pos1_rsq.width + h_width_cbar, pos1_rsq.y0, colorbar_width, fig_height]) 
#     cbar_rsq = fig.colorbar(cax_mean_online3,cax = cbaxes_rsq, ax=axes[1,0], shrink=1.0,ticks=flux_ticks_colorbar)
    cbar_rsq = fig.colorbar(cax_mean_online3,cax = cbaxes_rsq, ax=axes[1,0], shrink=1.0,ticks=flux_ticks_colorbar)
    cbar_rsq.ax.tick_params(labelsize=fontsize) 
    cbar_rsq.set_label(r'[kg m$^{-1}$ sec$^{-2}$]',fontsize=fontsize)

    pos1_rsq = axes[1, 1].get_position() # get the original position 
    cbaxes_rsq = fig.add_axes([pos1_rsq.x0+pos1_rsq.width + h_width_cbar, pos1_rsq.y0, colorbar_width, fig_height]) 
    cbar_rsq = fig.colorbar(cax_mean_online_enrgy,cax = cbaxes_rsq, ax=axes[1,1], shrink=1.0,ticks=flux_ticks_colorbarT)
    cbar_rsq.ax.tick_params(labelsize=fontsize) 
#     cbar_rsq.set_label(r'[kg m$^{-2}$ sec$^{-1}$ K]',fontsize=fontsize)
    if flag_moisture:
        cbar_rsq.set_label(r'[kg m$^{-2}$ sec$^{-1}$ kg$^{-1}$ kg]',fontsize=fontsize)
    else:
        cbar_rsq.set_label(r'[kg m$^{-2}$ sec$^{-1}$ K]',fontsize=fontsize)

    # pos1_rsq = axes[0, 1].get_position() # get the original position 
    # cbaxes_rsq = fig.add_axes([pos1_rsq.x0+pos1_rsq.width + h_width_cbar, pos1_rsq.y0, colorbar_width, fig_height]) 
    # cbar_rsq = fig.colorbar(cax_mean_online3,cax = cbaxes_rsq, ax=axes[0,1], shrink=1.0,)

    # pos1_rsq = axes[1, 1].get_position() # get the original position 
    # cbaxes_rsq = fig.add_axes([pos1_rsq.x0+pos1_rsq.width + h_width_cbar, pos1_rsq.y0, colorbar_width, fig_height]) 
    # cbar_rsq = fig.colorbar(cax_mean_online4,cax = cbaxes_rsq, ax=axes[1,1], shrink=1.0,)





    # pos1_rsq = axes[1, 0].get_position() # get the original position 
    # cbaxes_rsq = fig.add_axes([pos1_rsq.x0+pos1_rsq.width + h_width_cbar, pos1_rsq.y0, colorbar_width, fig_height]) 
    # cbar_rsq = fig.colorbar(cax_mean_online5,cax = cbaxes_rsq, ax=axes[1,0], shrink=1.0,)





    axes[1, 0].set_xlabel('Time (hours)',fontsize=fontsize)
    axes[1, 1].set_xlabel('Time (hours)',fontsize=fontsize)
    axes[0, 1].set_ylabel(r'Precipitation  [mm day$^{-1}$]',fontsize=fontsize)
    axes[0, 0].set_ylabel('Pressure [hPa]',fontsize=fontsize)
    axes[1, 0].set_ylabel('Pressure [hPa]',fontsize=fontsize)
    axes[1, 1].set_ylabel('Pressure [hPa]',fontsize=fontsize)


    # axes[0, 0].set_yticks(ticks_pmean)
    axes[0, 1].set_yticklabels([])
    # axes[1, 1].set_yticklabels([])


    
    axes[0,1].spines["top"].set_visible(False)
    axes[0,1].spines["right"].set_visible(False)

#     precip_ticks = [0,50,100,150,200,250,300,350,400,450]  
    axes[0, 1].set_ylim(0,2.6*precip_labels[-1])
    
#     ax2.set_ylim(-7000,8000)
    ax2.set_ylim(y_lim1,y_lim2)
#     ax2.set_ylim(-0.3000,0.3400)
#     precip_labels = [0,50,100,150]
    precip_ticks = precip_labels
    axes[0, 1].set_yticks(precip_ticks)
    axes[0, 1].set_yticklabels(precip_labels,fontsize=fontsize)

#     flux_ticks = [-2000,0,2000,4000,6000,8000]
#     flux_ticks = [-6000,-3000,0,3000,6000]
#     flux_ticks_label = [0,0.15000,0.3000]
#     flux_ticks_label = [-0.3,-0.15,0,0.15000,0.3000]
    flux_ticks = flux_ticks_label
    ax2.set_yticks(flux_ticks_label)
    ax2.set_yticklabels(flux_ticks_label,fontsize=fontsize)

    pressure_ticks = [300,600,900]
    time_ticks = [0,120,240]
    axes[0, 0].set_xticks(time_ticks)
    axes[0, 0].set_xticklabels([])
    axes[1, 0].set_xticks(time_ticks)
    axes[1, 0].set_xticklabels([])
    axes[1, 1].set_xticks(time_ticks)
    axes[1, 1].set_xticklabels(time_ticks,fontsize=fontsize)
    axes[0, 1].set_xticks(time_ticks)
    axes[0, 1].set_xticklabels(time_ticks,fontsize=fontsize)

    
    axes[0, 0].set_yticks(pressure_ticks)
    axes[0, 0].set_yticklabels(pressure_ticks,fontsize=fontsize)
    axes[1, 1].set_yticks(pressure_ticks)
    axes[1, 1].set_yticklabels(pressure_ticks,fontsize=fontsize)
    axes[1, 0].set_yticks(pressure_ticks)
    axes[1, 0].set_yticklabels(pressure_ticks,fontsize=fontsize)

    fig.delaxes(axes[2,0])
    fig.delaxes(axes[2,1])

    save_path = '/glade/work/janniy/data_for_offline_momentum_paper/plots/'
    if save_flag:
        plt.savefig(save_path+save_name1 + '.eps', format='eps', bbox_inches="tight")
    
#     plt.savefig(save_path+'snapshot_u_flux_precip.eps', format='eps', bbox_inches="tight")
#     plt.savefig(save_path+'snapshot_u_flux_T_flux_show_noise2.eps', format='eps', bbox_inches="tight")
    # plt.savefig(save_path+'correcting_u_precip_subgrid_hovemoller_tropics2_int_flux.eps', format='eps', bbox_inches="tight")

In [127]:
def plot_sapshots_x_trop(true_field,pred_field,true_field_int_z,pred_field_int_z,precip,p,y_i =80,change_range=0,save_flag = False,save_name1= 'del_snap'):
    per_day = 3600*24
    fontsize = 15
    fontsize_legend = 8
    h_width = 0.1
    v_width = 0.1
    h_width_cbar = 0.02
    colorbar_width =0.02

    row_dim = 3
    col_dim = 2

    fig_witdth = 1.
    fig_height = 0.75
    fig_h_dist = 0.2
    fig_v_dist = 0.2
    
    flux_ticks_colorbar = np.arange(-0.6,0.65,0.2)

    letters = ["(a) True flux","(b) y_i = 43","(b) Predicted flux","(d)",'(c) Integrated fluxes and precipitation','','','','','']

    size1 = 2
    fig, axes = plt.subplots(row_dim,col_dim)
    # fig.set_figheight(10)
    # fig.set_figwidth(10)
    upper_level_p = 47
    lowest_level = 0
    cmap1 = 'bwr'
    time_steps = 100
    time = np.arange(0,time_steps*3,3)
    n=20 
    yind1 = y_i
    x_ind1 = 10
    # wnumber = 1
    range11 = np.max(np.abs((true_u_flux[:,-time_steps:,yind1,x_ind1])))/2.5
    levels = np.linspace(-range11, range11, n+1)
    cax_mean_online2 =axes[0,0].contourf(time,p[lowest_level:upper_level_p],true_field[lowest_level:upper_level_p,-time_steps:,yind1,x_ind1],levels = levels,cmap=cmap1, extend="both")
    axes[0, 0].set_ylim(np.amax(p[lowest_level]), np.amin(p))

    if change_range == 1:
        range11 = np.max(np.abs((pred_field[:,-time_steps:,yind1,x_ind1])))/2.5
        levels = np.linspace(-range11, range11, n+1)
        upper_level_p = pred_field.shape[0]
    cax_mean_online3 =axes[1,0].contourf(time,p[lowest_level:upper_level_p],pred_field[lowest_level:upper_level_p,-time_steps:,yind1,x_ind1],levels = levels,cmap=cmap1, extend="both")
    axes[1, 0].set_ylim(np.amax(p[lowest_level]), np.amin(p))

    


    color = 'black'
    lns1 =axes[2,0].plot(time,precip[yind1,x_ind1,-time_steps:], color=color,label='precipitation')
    axes[2, 0].set_xlim(0, time[-1])
    axes[2, 0].tick_params(axis='y', labelcolor=color)


    ax2 = axes[2, 0].twinx()
    color = 'tab:blue'
    ax2.set_ylabel(r'Subgrid flux [kg m$^{-1}$ s$^{-2}$]', color=color,fontsize=fontsize)  # we already handled the x-label with ax1
    lns2 =ax2.plot(time,true_field_int_z[-time_steps:,yind1,x_ind1], color=color,label='integrated flux (true)')
    ax2.tick_params(axis='y', labelcolor=color)
    lns3 =ax2.plot(time,pred_field_int_z[-time_steps:,yind1,x_ind1], 'r:',label='integrated flux (predicted)')
    
    # added these three lines
    lns = lns1+lns2+lns3
    labs = [l.get_label() for l in lns]
    axes[2, 0].legend(lns, labs, loc=0,frameon=False)
    
    # yind1 = 43
    # # wnumber = 1
    # range11 = np.max(np.abs((fuz[:,yind1,x_ind1,:])))/1.4
    # levels = np.linspace(-range11, range11, n+1)
    # cax_mean_online3 =axes[0,1].contourf(time,p[lowest_level:upper_level_p],(fuz[lowest_level:upper_level_p,yind1,x_ind1,:]),levels = levels,cmap=cmap1)
    # axes[0, 1].set_ylim(np.amax(p[lowest_level]), np.amin(p))


    # color = 'tab:red'
    # cax_mean_online4 =axes[1,1].plot(time,precip11[yind1,x_ind1,:]*24*3600, color=color)
    # axes[1, 1].set_xlim(0, time[-1])
    # axes[1, 1].tick_params(axis='y', labelcolor=color)


    # ax2 = axes[1, 1].twinx()
    # color = 'tab:blue'
    # ax2.set_ylabel('subg flux', color=color)  # we already handled the x-label with ax1
    # cax_mean_online4 =ax2.plot(time,fuz_int[yind1,x_ind1,:], color=color)
    # ax2.tick_params(axis='y', labelcolor=color)



    x0 = axes[0,0].get_position().x0
    x1 = axes[0,0].get_position().x1
    y0 = axes[0,0].get_position().y0
    y1 = axes[0,0].get_position().y1

    for i in range(row_dim):
        for j  in range(col_dim):
            axes[i,j].set_position([x0 + j*fig_witdth + j*fig_h_dist,y0- i*fig_height - i*fig_v_dist,fig_witdth,fig_height])


    ind_let = 0
    for i in range(row_dim):
        for j  in range(col_dim):
            atext2 = axes[i, j].text(axes[0,0].get_position().x0*1.1+ j*fig_witdth + j*fig_h_dist,axes[0,0].get_position().y0*0.76- (i-1)*fig_height - (i-1)*fig_v_dist ,letters[ind_let],transform = fig.transFigure,color="black",fontsize=fontsize)

    #         header= axes[i, j].text(axes[0,0].get_position().x0*1.5+ j*fig_witdth + j*fig_h_dist,axes[0,0].get_position().y0*0.92- (i-1)*fig_height - (i-1)*fig_v_dist ,headers[ind_let],transform = fig.transFigure,color="black",fontsize=fontsize)
            ind_let = ind_let + 1

    pos1_rsq = axes[0, 0].get_position() # get the original position 
    cbaxes_rsq = fig.add_axes([pos1_rsq.x0+pos1_rsq.width + h_width_cbar, pos1_rsq.y0, colorbar_width, fig_height]) 
    cbar_rsq = fig.colorbar(cax_mean_online2,cax = cbaxes_rsq, ax=axes[0,0], shrink=1.0,ticks=flux_ticks_colorbar)
    cbar_rsq.ax.tick_params(labelsize=fontsize) 
    cbar_rsq.set_label(r'[kg m$^{-1}$ sec$^{-2}$]',fontsize=fontsize)
    
    if change_range == 1:
        flux_ticks_colorbar = np.arange(-range11,range11,range11/10)
    pos1_rsq = axes[1, 0].get_position() # get the original position 
    cbaxes_rsq = fig.add_axes([pos1_rsq.x0+pos1_rsq.width + h_width_cbar, pos1_rsq.y0, colorbar_width, fig_height]) 
#     cbar_rsq = fig.colorbar(cax_mean_online3,cax = cbaxes_rsq, ax=axes[1,0], shrink=1.0,ticks=flux_ticks_colorbar)
    cbar_rsq = fig.colorbar(cax_mean_online3,cax = cbaxes_rsq, ax=axes[1,0], shrink=1.0,ticks=flux_ticks_colorbar)
    cbar_rsq.ax.tick_params(labelsize=fontsize) 
    cbar_rsq.set_label(r'[kg m$^{-1}$ sec$^{-2}$]',fontsize=fontsize)



    # pos1_rsq = axes[0, 1].get_position() # get the original position 
    # cbaxes_rsq = fig.add_axes([pos1_rsq.x0+pos1_rsq.width + h_width_cbar, pos1_rsq.y0, colorbar_width, fig_height]) 
    # cbar_rsq = fig.colorbar(cax_mean_online3,cax = cbaxes_rsq, ax=axes[0,1], shrink=1.0,)

    # pos1_rsq = axes[1, 1].get_position() # get the original position 
    # cbaxes_rsq = fig.add_axes([pos1_rsq.x0+pos1_rsq.width + h_width_cbar, pos1_rsq.y0, colorbar_width, fig_height]) 
    # cbar_rsq = fig.colorbar(cax_mean_online4,cax = cbaxes_rsq, ax=axes[1,1], shrink=1.0,)





    # pos1_rsq = axes[1, 0].get_position() # get the original position 
    # cbaxes_rsq = fig.add_axes([pos1_rsq.x0+pos1_rsq.width + h_width_cbar, pos1_rsq.y0, colorbar_width, fig_height]) 
    # cbar_rsq = fig.colorbar(cax_mean_online5,cax = cbaxes_rsq, ax=axes[1,0], shrink=1.0,)





    axes[2, 0].set_xlabel('Time (hours)',fontsize=fontsize)
    axes[1, 1].set_xlabel('Time (hours)',fontsize=fontsize)
    axes[2, 0].set_ylabel(r'Precipitation  [mm day$^{-1}$]',fontsize=fontsize)
    axes[0, 0].set_ylabel('Pressure [hPa]',fontsize=fontsize)
    axes[1, 0].set_ylabel('Pressure [hPa]',fontsize=fontsize)


    # axes[0, 0].set_yticks(ticks_pmean)
    axes[0, 1].set_yticklabels([])
    # axes[1, 1].set_yticklabels([])


    
    axes[2,0].spines["top"].set_visible(False)
    axes[2,0].spines["right"].set_visible(False)

#     precip_ticks = [0,50,100,150,200,250,300,350,400,450]  
    axes[2, 0].set_ylim(0,400)
    ax2.set_ylim(-2000,3000)
    precip_ticks_label = [0,20,40,60]
    precip_ticks = precip_ticks_label
    axes[2, 0].set_yticks(precip_ticks)
    axes[2, 0].set_yticklabels(precip_ticks_label,fontsize=fontsize)

#     flux_ticks = [-2000,0,2000,4000,6000,8000]
#     flux_ticks = [-6000,-3000,0,3000,6000]
    flux_ticks_label = [0,1000,2000]
    flux_ticks = flux_ticks_label
    ax2.set_yticks(flux_ticks_label)
    ax2.set_yticklabels(flux_ticks_label,fontsize=fontsize)

    pressure_ticks = [300,600,900]
    time_ticks = [0,120,240]
    axes[0, 0].set_xticks(time_ticks)
    axes[0, 0].set_xticklabels([])
    axes[1, 0].set_xticks(time_ticks)
    axes[1, 0].set_xticklabels([])
    axes[2, 0].set_xticks(time_ticks)
    axes[2, 0].set_xticklabels(time_ticks,fontsize=fontsize)

    axes[0, 0].set_yticks(pressure_ticks)
    axes[0, 0].set_yticklabels(pressure_ticks,fontsize=fontsize)
    axes[1, 0].set_yticks(pressure_ticks)
    axes[1, 0].set_yticklabels(pressure_ticks,fontsize=fontsize)

    fig.delaxes(axes[0,1])
    fig.delaxes(axes[1,1])
    fig.delaxes(axes[2,1])

    save_path = '/glade/work/janniy/data_for_offline_momentum_paper/plots/'
    if save_flag:
        plt.savefig(save_path+save_name1 + '.eps', format='eps', bbox_inches="tight")
    
#     plt.savefig(save_path+'snapshot_u_flux_precip.eps', format='eps', bbox_inches="tight")
#     plt.savefig(save_path+'snapshot_u_flux_T_flux_show_noise2.eps', format='eps', bbox_inches="tight")
    # plt.savefig(save_path+'correcting_u_precip_subgrid_hovemoller_tropics2_int_flux.eps', format='eps', bbox_inches="tight")